# Gemini 2.0 - Multimodal live API and MongoDB Atlas Vector store as tools

Inspired and built on top of the following Google [example notebook](https://github.com/google-gemini/cookbook/blob/main/gemini-2/live_api_tool_use.ipynb).

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mongodb-developer/GenAI-Showcase/blob/main/notebooks/agents/Gemini2_0_multi_modality_with_mongodb_atlas_vector_store.ipynb)

This notebook provides examples of how to use tools with the multimodal live API with [Gemini 2.0](https://ai.google.dev/gemini-api/docs/models/gemini-v2) and [MongoDB Atlas with langchain integration](https://www.mongodb.com/docs/atlas/atlas-vector-search/ai-integrations/langchain/) as tools.

The tutorial build an agentic multimodal agent in websocket realtime API to fetch and store MongoDB context documents. It uses Function Calling tools. The earlier Gemini models supported versions of these tools. The biggest change with Gemini 2 (in the Live API) is that, basically, all the tools are handled by Code Execution. With that change, you can use **multiple tools** in a single API call.  

This tutorial assumes you are familiar with the Live API, as described in the [this tutorial](https://github.com/google-gemini/cookbook/blob/main/gemini-2/live_api_starter.ipynb).

## Setup

### Install SDK

The new **[Google Gen AI SDK](https://ai.google.dev/gemini-api/docs/sdks)** provides programmatic access to Gemini 2.0 (and previous models) using both the [Google AI for Developers](https://ai.google.dev/gemini-api/docs) and [Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/overview) APIs. With a few exceptions, code that runs on one platform will run on both. This means that you can prototype an application using the Developer API and then migrate the application to Vertex AI without rewriting your code.

More details about this new SDK on the [documentation](https://ai.google.dev/gemini-api/docs/sdks) or in the [Getting started](../gemini-2/get_started.ipynb) notebook.

In [1]:
!pip install -U -q google-genai

### Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](../quickstarts/Authentication.ipynb) for an example.

In [4]:
from google.colab import userdata
import os
import getpass


os.environ['GOOGLE_API_KEY'] = getpass.getpass("Input your Google API Key")

Input your Google API Key··········


### Initialize SDK client

The client will pickup your API key from the environment variable.
To use the live API you need to set the client version to `v1alpha`.

In [5]:
from google import genai

client = genai.Client(http_options= {
      'api_version': 'v1alpha'
})

### Select a model

Multimodal Live API are a new capability introduced with the [Gemini 2.0](https://ai.google.dev/gemini-api/docs/models/gemini-v2) model. It won't work with previous generation models.

In [6]:
model_name = "gemini-2.0-flash-exp"

### Imports

In [7]:
import asyncio
import contextlib
import json
import wave

from IPython import display

from google import genai
from google.genai import types

### Utilities

You're going to use the Live API's audio output, the easiest way hear it in Colab is to write the `PCM` data out as a `WAV` file:

In [8]:
@contextlib.contextmanager
def wave_file(filename, channels=1, rate=24000, sample_width=2):
    with wave.open(filename, "wb") as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(sample_width)
        wf.setframerate(rate)
        yield wf

Use a logger so it's easier to switch on/off debugging messages.

In [9]:
import logging
logger = logging.getLogger('Live')
#logger.setLevel('DEBUG')  # Switch between "INFO" and "DEBUG" to toggle debug messages.
logger.setLevel('INFO')

## Get started

Most of the Live API setup will be similar to the [starter tutorial](../gemini-2/live_api_starter.ipynb). Since this tutorial doesn't focus on the realtime interactivity of the API, the code has been simplified: This code uses the Live API, but it only sends a single text prompt, and listens for a single turn of replies.

You can set `modality="AUDIO"` on any of the examples to get the spoken version of the output.

In [56]:
n = 0
async def run(prompt, modality="AUDIO", tools=None):
  global n
  if tools is None:
    tools=[]

  config = {
          "tools": tools,
          "system_instruction" : "You are a helpful HR assistant who can search employees with atlas_search_tool and create teams in the database with create_team tool",
          "generation_config": {
              "response_modalities": [modality]}}
  print(f"before client invoke {tools}")
  async with client.aio.live.connect(model=model_name, config=config) as session:
    display.display(display.Markdown(prompt))
    display.display(display.Markdown('-------------------------------'))
    await session.send(prompt, end_of_turn=True)

    audio = False
    filename = f'audio_{n}.wav'
    with wave_file(filename) as wf:
      async for response in session.receive():
        logger.debug(str(response))
        if text:=response.text:
          display.display(display.Markdown(text))
          continue

        if data:=response.data:
          print('.', end='')
          wf.writeframes(data)
          audio = True
          continue

        server_content = response.server_content
        if server_content is not None:
          handle_server_content(wf, server_content)
          continue
        print(f"Before tool call {response.tool_call}")

        tool_call = response.tool_call
        if tool_call is not None:
          await handle_tool_call(session, tool_call)


  if audio:
    display.display(display.Audio(filename, autoplay=True))
    n = n+1

Since this tutorial demonstrates several tools, you'll need more code to handle the different types of objects it returns.

For example:

- The `code_execution` tool can return `executable_code` and `code_execution_result` parts.
- The `google_search` tool may attach a `grounding_metadata` object.

In [11]:
def handle_server_content(wf, server_content):
  model_turn = server_content.model_turn
  if model_turn:
    for part in model_turn.parts:
      executable_code = part.executable_code
      if executable_code is not None:
        display.display(display.Markdown('-------------------------------'))
        display.display(display.Markdown(f'``` python\n{executable_code.code}\n```'))
        display.display(display.Markdown('-------------------------------'))

      code_execution_result = part.code_execution_result
      if code_execution_result is not None:
        display.display(display.Markdown('-------------------------------'))
        display.display(display.Markdown(f'```\n{code_execution_result.output}\n```'))
        display.display(display.Markdown('-------------------------------'))

  grounding_metadata = getattr(server_content, 'grounding_metadata', None)
  if grounding_metadata is not None:
    display.display(
        display.HTML(grounding_metadata.search_entry_point.rendered_content))

  return

- Finally, with the `function_declarations` tool, the API may return `tool_call` objects. In our case we will have 2 MongoDB tools
- `atlas_search_tool` :  Search employee records using Atlas Vector search for semantic similarity
- `create_team` :  A tool that writes a record with a team name and a people array with assigned names as the array strings.

In [36]:
import json
async def handle_tool_call(session, tool_call):
  for fc in tool_call.function_calls:
    function_name = fc.name
    arguments = fc.args
    if function_name == "create_team":
      team = arguments.get("team_data")
      result = create_team(team.get('name'), team.get('people'))
    elif function_name == "atlas_search_tool":
      result = atlas_search(arguments.get("query"), arguments.get("k", 5))
    else:
      result = "Unknown function"
    tool_response = types.LiveClientToolResponse(
        function_responses=[types.FunctionResponse(
            name=fc.name,
            id=fc.id,
            response={'result': result},
        )]
    )

    print('\n>>> ', tool_response)
    await session.send(tool_response)

Try running it for a first time with no tools:

In [13]:
await run(prompt="Hello?", tools=None, modality = "AUDIO")

before client invoke []


Hello?

-------------------------------

.......

## Atlas function setup and calls

### MongoDB Vector Database and Connection Setup

MongoDB acts as both an operational and a vector database for the RAG system.
MongoDB Atlas specifically provides a database solution that efficiently stores, queries and retrieves vector embeddings.

Creating a database and collection within MongoDB is made simple with MongoDB Atlas.

1. First, register for a [MongoDB Atlas account](https://www.mongodb.com/cloud/atlas/register). For existing users, sign into MongoDB Atlas.
2. [Follow the instructions](https://www.mongodb.com/docs/atlas/tutorial/deploy-free-tier-cluster/). Select Atlas UI as the procedure to deploy your first cluster.

In [14]:
# prompt: add mongodb depndencies

!pip install pymongo langchain-google-genai langchain-core langchain-mongodb langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


# Prepare MongoDB Atlas vector store

Run the following code to create the Atlas Vector Search index and insert some vectorised employee records for our database.

In [15]:
from pymongo import MongoClient
from google.api_core import retry
from bson import json_util
from pymongo.operations import SearchIndexModel
import json
import time

# Replace with your MongoDB connection string
MONGO_URI = getpass.getpass("Input your MongoDB Atlas URI:")

# Define the database and collections
mongoClient = MongoClient(MONGO_URI, appname="devrel.showcase.gemini20_agent")
db = mongoClient["google-ai"]
collection = db["embedded_docs"]

db.create_collection("embedded_docs")

# Create the search index
## create index
search_index_model = SearchIndexModel(
  definition={
    "fields": [
      {
        "type": "vector",
        "numDimensions": 768,
        "path": "embedding",
        "similarity": "cosine"
      },
    ]
  },
  name="vector_index",
  type="vectorSearch",
)
result = collection.create_search_index(model=search_index_model)
print("New search index named " + result + " is building.")
# Wait for initial sync to complete
print("Polling to check if the index is ready. This may take up to a minute.")
predicate=None
if predicate is None:
  predicate = lambda index: index.get("queryable") is True
while True:
  indices = list(collection.list_search_indexes(result))
  if len(indices) and predicate(indices[0]):
    break
  time.sleep(5)
print(result + " is ready for querying.")





Input your MongoDB Atlas URI:··········
New search index named vector_index is building.
Polling to check if the index is ready. This may take up to a minute.
vector_index is ready for querying.


In [ ]:
## Insert Employee Data

In this section, we will insert sample employee data into the MongoDB Atlas vector store. This data includes employee details such as name, department, location, and salary, along with their respective embeddings.

In [16]:
## Insert data

collection.insert_many([{
  "_id": "54634",
  "content": "Employee number 54634, name  Jane Doe, department  Marketing, location  Los Angeles, salary  120000",
  "embedding": [
    0.024926867,
    -0.049224764,
    0.0051397122,
    -0.015662413,
    0.036198545,
    0.020058708,
    0.07437574,
    -0.023353964,
    0.009316206,
    0.010908616,
    -0.022639172,
    0.008110297,
    -0.03569339,
    0.016980717,
    -0.014814842,
    0.0048693726,
    0.0024207153,
    -0.036100663,
    -0.016500184,
    -0.033307776,
    -0.020310277,
    -0.01708344,
    -0.017491976,
    -0.01000457,
    0.021011023,
    -0.0017388392,
    0.00891552,
    -0.10860842,
    -0.046374027,
    -0.01210933,
    -0.043089807,
    0.027616654,
    -0.058572993,
    -0.0012424898,
    -0.0009245786,
    -0.026917346,
    -0.026614873,
    -0.008031103,
    0.006364708,
    0.022180663,
    -0.029214343,
    -0.020451233,
    -0.013976919,
    -0.011516259,
    0.027531886,
    -0.020989226,
    0.0011997295,
    -0.008541397,
    0.013981253,
    -0.09130217,
    0.031902086,
    -0.014483433,
    0.04141627,
    -0.022633772,
    -0.0015243818,
    -0.0701282,
    -0.005745007,
    0.003046663,
    -0.00138343,
    -0.0483541,
    -0.018663412,
    -0.010342808,
    -0.036891118,
    0.041526485,
    -0.0070978166,
    -0.056960497,
    -0.00027713762,
    0.00041085767,
    0.0638381,
    0.012412274,
    -0.042297978,
    -0.034797642,
    0.027877614,
    -0.014577787,
    -0.07915758,
    -0.11489053,
    -0.012170335,
    0.023879664,
    0.040547226,
    0.027829757,
    -0.019437442,
    -0.03378374,
    -0.026225261,
    -0.042423252,
    -0.034459304,
    0.07092275,
    0.04282131,
    -0.019700523,
    -0.022706546,
    0.07524933,
    -0.025327584,
    0.03845969,
    -0.006575861,
    -0.05432576,
    -0.0030720695,
    0.06054884,
    0.014463371,
    -0.05177936,
    0.011856342,
    -0.01622149,
    -0.015594266,
    -0.023064198,
    -0.056572527,
    -0.01887025,
    0.009372615,
    -0.005908454,
    -0.019093627,
    -0.010138043,
    -0.03215679,
    0.023405561,
    -0.07793633,
    0.0022705605,
    -0.03748151,
    -0.040298775,
    0.05949639,
    -0.02171212,
    -0.05205988,
    0.056799524,
    0.046728708,
    0.00995629,
    0.018814746,
    0.052283265,
    0.0133213885,
    0.0070853233,
    0.02381203,
    0.048278138,
    -0.025068594,
    0.013948586,
    0.10241069,
    0.033334825,
    -0.035002332,
    -0.028427497,
    0.036363285,
    -0.009638689,
    0.050988846,
    0.088660076,
    0.052428994,
    0.008259064,
    0.051591944,
    -0.035510417,
    -0.0025276055,
    0.020777041,
    -0.02206114,
    0.00075541,
    -0.038383663,
    0.0068223546,
    0.013984699,
    -0.04017368,
    0.046198152,
    -0.015898008,
    -0.016150242,
    -0.006470939,
    -0.046308447,
    0.033918925,
    0.021597652,
    0.009154935,
    -0.03465381,
    0.0016349686,
    0.019491052,
    0.023978025,
    0.059030097,
    0.03193792,
    -0.026359702,
    0.025488824,
    0.0014710033,
    0.021635707,
    0.028962605,
    -0.0426874,
    -0.025902368,
    -0.03850525,
    -0.0072347843,
    0.019107448,
    -0.039649457,
    -0.04149649,
    -0.0390399,
    -0.073608436,
    0.005187664,
    -0.012122672,
    -0.02703804,
    -0.025964485,
    -0.041734103,
    -0.029006453,
    0.020049337,
    0.04200867,
    -0.0018111905,
    0.03557779,
    0.058680393,
    -0.045999996,
    0.0054121087,
    -0.028696584,
    0.036451954,
    -0.00065523863,
    -0.014693241,
    -0.0049391747,
    -0.013985914,
    0.029479476,
    0.005661245,
    0.034519162,
    -0.0017001618,
    -0.014198862,
    -0.044131663,
    0.04520943,
    0.021466821,
    -0.0052906116,
    0.024838196,
    -0.036314182,
    0.06807027,
    -0.046411086,
    0.012543244,
    -0.0015524302,
    -0.005904932,
    0.03395419,
    -0.012026594,
    -0.030101227,
    0.028219236,
    0.030846382,
    0.004014791,
    0.0011687118,
    -0.005924506,
    -0.045966923,
    0.011490533,
    0.0070699626,
    -0.04636452,
    0.051188413,
    -0.010941066,
    -0.0003843776,
    -0.03554462,
    0.031527005,
    0.009976034,
    -0.07566613,
    0.009484453,
    0.075585775,
    0.036681112,
    -0.01997137,
    0.04221241,
    0.028756386,
    -0.0051457584,
    0.017223062,
    -0.026450416,
    -0.0031493392,
    -0.031142443,
    0.014129824,
    0.030640334,
    0.029624552,
    -0.028791834,
    0.014263747,
    -0.052234437,
    0.016012207,
    0.0054665036,
    0.049960792,
    0.01722948,
    -0.045383565,
    0.035611328,
    0.017666759,
    -0.034179296,
    0.029283382,
    -0.0925589,
    0.005352651,
    0.015380154,
    -0.025827546,
    0.078813694,
    0.023472652,
    -0.015502742,
    -0.0109618185,
    -0.062091105,
    0.020872923,
    0.0000086566615,
    -0.046515323,
    -0.010241027,
    0.07056774,
    -0.0039463,
    -0.0049326513,
    0.08365995,
    0.04134926,
    0.028786693,
    0.003118606,
    -0.00762385,
    0.0374987,
    0.011105095,
    -0.063480176,
    0.0057922564,
    0.031258017,
    0.016701866,
    -0.028914252,
    -0.003122066,
    0.0040146816,
    -0.042392317,
    0.007909141,
    -0.0074412455,
    -0.061981037,
    -0.0753461,
    -0.017482404,
    0.0019353802,
    -0.039474253,
    -0.03557083,
    0.02512347,
    -0.03232185,
    0.029252835,
    0.0036022866,
    -0.0006892634,
    0.020094976,
    -0.038219,
    -0.017786479,
    -0.08057002,
    0.031012703,
    0.0027172887,
    0.008962194,
    -0.052723087,
    -0.010544319,
    -0.015351896,
    0.032455456,
    -0.018037044,
    -0.031501703,
    -0.00675466,
    0.0221693,
    0.045492973,
    -0.0019718895,
    -0.036067158,
    -0.019214695,
    0.092174225,
    0.0011163659,
    0.020685544,
    0.042855497,
    -0.037252106,
    0.0040059835,
    0.01912792,
    -0.015698344,
    0.04483261,
    -0.004036565,
    0.034471065,
    -0.00026268934,
    0.021624925,
    -0.029354827,
    0.047258046,
    0.00198258,
    0.030219225,
    -0.070994265,
    -0.012291296,
    -0.06872952,
    -0.01746057,
    0.051459547,
    -0.005943351,
    -0.05806519,
    -0.049041413,
    -0.0029545315,
    -0.03594199,
    -0.02741998,
    0.023392593,
    0.06987801,
    0.03409129,
    0.010657495,
    0.029227218,
    -0.016149484,
    -0.033026453,
    -0.026461514,
    -0.029547209,
    0.052853283,
    -0.005257883,
    0.019270778,
    -0.021651607,
    -0.0679393,
    0.022915237,
    -0.018916594,
    0.020382203,
    0.0061164782,
    -0.0053500766,
    0.04022389,
    0.010382376,
    0.0037303683,
    0.00038739407,
    0.043591883,
    -0.050327547,
    -0.0032173207,
    0.02382172,
    -0.018193057,
    -0.049482215,
    -0.034972608,
    -0.03591731,
    0.055168044,
    0.04351997,
    0.042791825,
    0.014279212,
    0.007098134,
    0.041995537,
    0.02505999,
    -0.014369368,
    0.022210745,
    0.035329152,
    -0.0018724868,
    0.04496197,
    -0.015458536,
    -0.035687756,
    0.11576407,
    0.014072529,
    0.01368354,
    -0.044601526,
    -0.017274825,
    -0.035651878,
    0.018606395,
    -0.02624995,
    -0.008605139,
    -0.05559558,
    -0.031822406,
    -0.0025050528,
    -0.007260074,
    0.008371022,
    -0.009121923,
    -0.010686996,
    -0.004175405,
    -0.07709291,
    0.02095484,
    0.061555583,
    0.05423064,
    -0.08377491,
    -0.044701204,
    -0.022143545,
    0.03825245,
    0.020806536,
    -0.025773121,
    0.026748922,
    -0.035925206,
    0.007971478,
    -0.07475945,
    -0.067393735,
    -0.01523033,
    -0.025753228,
    0.0007241217,
    -0.016581649,
    -0.017368209,
    0.008112616,
    -0.008466575,
    -0.009379433,
    0.013422547,
    0.010592169,
    -0.0036504674,
    -0.041012507,
    -0.018081397,
    0.005491686,
    -0.02313517,
    0.010353996,
    0.06254845,
    -0.029144084,
    0.07706289,
    -0.002867055,
    -0.022228312,
    -0.028821543,
    0.00022086965,
    -0.05151184,
    0.0114417225,
    -0.10001401,
    -0.007430506,
    -0.0810789,
    -0.0106861945,
    -0.022859078,
    -0.011867412,
    -0.03224893,
    0.017619586,
    0.07519098,
    -0.0035126992,
    0.005702041,
    0.0050863526,
    -0.024752518,
    -0.051708452,
    -0.08042799,
    -0.0016043095,
    -0.054452665,
    0.04997996,
    -0.022929264,
    0.07225404,
    0.057153042,
    0.016426055,
    -0.04732747,
    0.015056493,
    0.036387,
    -0.011774074,
    -0.01798128,
    -0.018891433,
    0.027562132,
    0.026849896,
    -0.027375022,
    0.014004462,
    -0.04581184,
    0.062606476,
    -0.028271144,
    -0.03200168,
    0.01676934,
    0.005372289,
    -0.039633606,
    0.013623909,
    0.0483795,
    0.025562169,
    -0.0007961121,
    -0.010983261,
    -0.03702811,
    -0.020723386,
    0.01622944,
    0.028984541,
    0.052704614,
    0.026976695,
    0.040055428,
    -0.027315127,
    0.022882,
    -0.004998423,
    0.005663411,
    0.05058019,
    0.025238585,
    -0.0043412056,
    -0.010151074,
    -0.005550237,
    0.012124635,
    -0.012980639,
    -0.030452866,
    -0.009688401,
    0.0011759287,
    0.05065029,
    -0.034934316,
    0.00009035412,
    0.0016641455,
    0.038797174,
    -0.041665185,
    0.02898577,
    0.0050686207,
    -0.11780856,
    -0.017124109,
    0.058547672,
    -0.047212645,
    0.019724954,
    0.052281905,
    -0.028578363,
    0.041821305,
    -0.01702321,
    0.07445442,
    -0.026752556,
    -0.005502298,
    -0.030765718,
    0.018439377,
    -0.016507415,
    0.053340096,
    0.027617462,
    0.0028326863,
    0.043026447,
    -0.036896102,
    0.015555754,
    -0.028502347,
    -0.027034258,
    0.024761003,
    0.06037164,
    -0.088444225,
    0.020785877,
    0.01979808,
    -0.024642324,
    0.030752076,
    0.04398454,
    -0.018832913,
    -0.019311974,
    0.027594192,
    -0.029102236,
    -0.009168705,
    -0.012181484,
    -0.023603536,
    -0.019860711,
    -0.0035978511,
    0.012397888,
    0.0011611527,
    0.0898995,
    0.02485896,
    -0.005481566,
    -0.039879583,
    0.022059798,
    -0.012275004,
    -0.014664887,
    0.026405737,
    0.006707709,
    0.0036560902,
    0.07902236,
    -0.037842136,
    -0.034537956,
    0.028122894,
    -0.010967483,
    0.002525521,
    0.06961038,
    0.016122231,
    -0.03932486,
    0.024220413,
    -0.042630956,
    0.02517927,
    0.054458976,
    0.013454574,
    0.06871976,
    -0.0023826372,
    -0.06773266,
    0.02460262,
    -0.062439863,
    -0.05781772,
    -0.045605063,
    0.027145335,
    -0.04323481,
    -0.027004242,
    0.027012022,
    0.0021861214,
    0.016324826,
    -0.0575887,
    0.0639403,
    -0.013034681,
    0.05562446,
    -0.02185531,
    0.025901046,
    -0.019938763,
    0.008652073,
    0.030600363,
    0.036625423,
    -0.001488325,
    -0.014308819,
    -0.035884213,
    -0.0443492,
    -0.018744895,
    0.012494876,
    0.009342181,
    -0.023556268,
    -0.024485394,
    0.008759082,
    -0.036470927,
    0.003922266,
    -0.005570674,
    0.06833909,
    0.0064327326,
    -0.018937796,
    -0.00621957,
    0.062171705,
    0.038591076,
    0.031692512,
    0.041171357,
    0.021134553,
    -0.0029191682,
    -0.03674039,
    0.024994813,
    -0.023660952,
    -0.055257946,
    0.040094994,
    -0.008852677,
    -0.087461375,
    0.01325573,
    0.040444717,
    -0.01021572,
    -0.026924513,
    0.034332998,
    0.03086733,
    -0.07686781,
    -0.016338205,
    -0.020646619,
    -0.042191442,
    0.009642032,
    0.027536497,
    0.022932088,
    -0.014337762,
    -0.035221837,
    -0.031101929,
    -0.050192293,
    0.04678706,
    0.0022375528,
    -0.019851457,
    0.05928813,
    -0.00034617726,
    0.02348867,
    -0.005559518,
    0.024877217,
    0.006464173,
    -0.061497923,
    -0.044427488,
    0.09275633,
    -0.060154688,
    0.035813265,
    0.008149022,
    -0.021311855,
    0.06460764,
    0.074633114,
    -0.055023994,
    0.008292285,
    0.0034740432,
    0.04366205,
    -0.019244209,
    -0.011739611,
    0.010764034,
    0.009531266,
    0.01438961,
    0.036831133,
    -0.014654368,
    0.025802445,
    -0.05894489,
    -0.024810959,
    -0.0048909825,
    -0.02695748,
    0.0062228707,
    -0.029853752,
    0.07839843,
    0.025475468,
    0.03605201,
    -0.013841105,
    0.034315526,
    0.05802323,
    -0.0065083285,
    -0.049404792,
    0.0047248784,
    0.04340827,
    -0.037954953,
    0.022516504,
    0.0039298353,
    0.00731798,
    0.03454248,
    -0.01497592,
    0.04957702,
    -0.056836925,
    -0.039613705,
    0.030464869,
    0.03466243,
    -0.0054599266,
    0.0375021,
    -0.0047780927,
    -0.04908544,
    0.030942274,
    0.055504274,
    -0.018186081,
    0.008902889,
    -0.02538345,
    -0.008923959,
    0.06603814,
    -0.026365338,
    0.06799129,
    -0.014765047,
    -0.019033851,
    0.06114885,
    -0.04529476,
    -0.03121909,
    0.035800617,
    -0.033105463,
    0.0592133,
    0.015604761,
    0.039122812,
    -0.009252219,
    -0.06183396,
    -0.04536972,
    0.035133872,
    0.028567217,
    0.028155535,
    0.044882603,
    -0.018319324,
    0.023039794,
    0.0067167506,
    -0.026577776,
    -0.058001574,
    -0.104534015,
    0.042269215,
    0.004363603,
    0.048697747,
    0.04533539,
    -0.05496062,
    -0.021759328,
    -0.004851495,
    0.03850219,
    0.020519726,
    -0.04097515,
    -0.001369751,
    -0.016953135,
    0.046057485,
    0.005478688,
    0.033872366,
    -0.01388759,
    0.026152821
  ]
},
{
  "_id": "54633",
  "content": "Employee number 54633, name  John Doe, department  Sales, location  New York, salary  100000",
  "embedding": [
    0.02363214,
    -0.064981155,
    -0.015693784,
    -0.0007628399,
    0.053643532,
    0.03410332,
    0.061779644,
    -0.018410124,
    0.00021801237,
    0.010404121,
    -0.034521878,
    -0.0018919241,
    -0.031763557,
    -0.021995055,
    -0.028728155,
    0.021817293,
    0.017098326,
    -0.023482114,
    0.00035919988,
    -0.030648591,
    -0.040644307,
    -0.022990027,
    -0.03393246,
    -0.006558424,
    0.02599438,
    -0.002243526,
    -0.005706434,
    -0.09497937,
    -0.041159507,
    -0.014462122,
    -0.07141966,
    0.036013767,
    -0.040423963,
    0.015345732,
    -0.010977615,
    -0.031959552,
    -0.04254595,
    0.0140362885,
    -0.0050170403,
    0.0062000453,
    -0.03828404,
    -0.0153707955,
    -0.02084843,
    -0.029029334,
    0.0488787,
    0.0116918655,
    -0.0008196994,
    0.02450069,
    0.012549454,
    -0.098332606,
    0.034913078,
    -0.011317091,
    0.05609082,
    -0.023273163,
    -0.018277766,
    -0.06916207,
    0.00826931,
    0.010393335,
    -0.01787084,
    -0.042722043,
    -0.011604089,
    -0.0012522464,
    -0.014621865,
    0.038314685,
    0.0005260519,
    -0.06758038,
    -0.018045135,
    0.010444268,
    0.07087152,
    -0.00284572,
    -0.037000503,
    -0.0025446592,
    0.04700479,
    -0.001956285,
    -0.052959703,
    -0.12006671,
    -0.0042574876,
    0.033431716,
    0.053164013,
    0.045192037,
    0.0069061965,
    -0.015518668,
    -0.030133313,
    -0.044831418,
    -0.03726597,
    0.057425573,
    0.038113765,
    -0.027094543,
    -0.007922181,
    0.04684854,
    -0.019562198,
    0.030295257,
    -0.004370621,
    -0.058695186,
    -0.018610591,
    0.061370887,
    0.030731026,
    -0.04589357,
    0.009940706,
    -0.01948416,
    -0.0046085874,
    0.0022432224,
    -0.037660263,
    0.0066369097,
    0.024275409,
    -0.00056191423,
    -0.000042045995,
    -0.011206171,
    -0.045394268,
    0.0071125193,
    -0.07760038,
    0.00075220404,
    -0.01073115,
    -0.04549924,
    0.06086503,
    -0.039449498,
    -0.058962233,
    0.06468895,
    0.04064492,
    0.0028264702,
    0.046224788,
    0.054499805,
    0.017846588,
    0.039633967,
    0.043515794,
    0.045659285,
    -0.008033441,
    0.017785944,
    0.110720046,
    0.04685865,
    -0.01802922,
    -0.040406503,
    0.046266943,
    -0.013695085,
    0.049763296,
    0.06897263,
    0.04518968,
    0.053687356,
    0.047708154,
    -0.034360345,
    -0.006671896,
    0.001714356,
    -0.04809813,
    -0.014786435,
    -0.017878355,
    -0.008214343,
    -0.0021909962,
    -0.046269346,
    0.072654426,
    -0.017648848,
    0.024896594,
    -0.007919613,
    -0.022969129,
    0.03022781,
    0.011041212,
    0.02506789,
    -0.042059314,
    -0.012793883,
    0.014914554,
    -0.0069050984,
    0.04926195,
    0.029791638,
    0.00091826794,
    0.022105714,
    0.02267874,
    0.01301686,
    0.041148506,
    -0.040438164,
    -0.017320365,
    -0.03499895,
    0.021354463,
    0.01690759,
    -0.025965609,
    -0.031534,
    -0.030494336,
    -0.07054347,
    -0.013133889,
    -0.012822187,
    -0.029168444,
    -0.0218087,
    -0.026540095,
    -0.03263288,
    0.0021044614,
    0.048010275,
    -0.01386612,
    0.038521104,
    0.04187871,
    -0.043820612,
    0.01064401,
    -0.02302523,
    0.036518324,
    0.02567038,
    -0.02029525,
    -0.008654853,
    0.006971086,
    0.031201571,
    0.025180522,
    0.04371456,
    0.003744122,
    -0.030606078,
    -0.057033766,
    0.07168329,
    0.008192757,
    0.0008544097,
    0.021805858,
    -0.03130309,
    0.07239574,
    -0.070142336,
    -0.008480367,
    -0.008728997,
    -0.0034903064,
    0.046792153,
    -0.023682427,
    -0.030346792,
    0.019847026,
    0.012278681,
    -0.0025996969,
    0.031041447,
    -0.0077332347,
    -0.038237624,
    0.012388913,
    0.0188294,
    -0.02149499,
    0.051257458,
    0.002782599,
    -0.010533314,
    0.0011125503,
    0.012332888,
    0.01806485,
    -0.06757561,
    0.0023377982,
    0.09027674,
    0.0307473,
    -0.0031313177,
    0.031805944,
    0.02137824,
    -0.015408471,
    0.00030692422,
    -0.020330938,
    0.023312671,
    -0.030245807,
    0.025795639,
    0.04197928,
    0.00660178,
    -0.040649693,
    0.024375524,
    -0.04237185,
    0.032011457,
    0.015374865,
    0.04028766,
    0.010178039,
    -0.03617627,
    0.047202107,
    0.020767882,
    -0.04539947,
    0.019767804,
    -0.10889575,
    0.017927451,
    0.017144594,
    -0.015266046,
    0.06631826,
    0.02412635,
    -0.024798216,
    -0.008509299,
    -0.0639002,
    0.0136915855,
    0.0027062744,
    -0.064476475,
    -0.0012653655,
    0.07816678,
    -0.012643878,
    -0.008585973,
    0.07736182,
    0.04327385,
    0.025656862,
    0.0030979763,
    0.00021765077,
    0.031746045,
    0.02362011,
    -0.04819947,
    -0.012916141,
    0.018860366,
    0.017766127,
    -0.023901632,
    0.004137154,
    0.0041561704,
    -0.0022801503,
    0.009764509,
    0.002656565,
    -0.04468994,
    -0.07757732,
    -0.015850794,
    0.010670362,
    -0.04420402,
    -0.039033562,
    0.0031982567,
    -0.04361803,
    0.034209713,
    0.005868743,
    0.032322675,
    0.039166275,
    -0.03746624,
    -0.021231664,
    -0.06653363,
    0.059266083,
    0.016893726,
    0.022510596,
    -0.06638812,
    -0.026510475,
    -0.011830923,
    0.0069003874,
    -0.043289896,
    -0.03288132,
    -0.011146353,
    0.0287577,
    0.04004958,
    -0.0014789595,
    -0.02158115,
    -0.012407836,
    0.073853076,
    0.021458639,
    0.020401636,
    0.039237395,
    -0.025395788,
    -0.005443788,
    0.030352518,
    -0.035680313,
    0.032063533,
    0.0016429706,
    0.01741445,
    0.0024685974,
    0.034620967,
    -0.010647702,
    0.0526942,
    -0.0034993906,
    0.03208014,
    -0.08731866,
    -0.015013707,
    -0.06679643,
    -0.015791807,
    0.033754732,
    -0.00432221,
    -0.049488492,
    -0.04113329,
    -0.025805045,
    -0.01941248,
    -0.027105281,
    -0.0048762094,
    0.063147336,
    0.041223466,
    0.028617661,
    0.021648958,
    -0.041316215,
    -0.0313366,
    -0.035062548,
    -0.033159826,
    0.040288758,
    0.0010510484,
    0.007258658,
    -0.032413058,
    -0.061028056,
    0.031196948,
    -0.021832671,
    0.014297596,
    0.01129684,
    -0.014241179,
    0.019869702,
    0.02111102,
    0.0012747784,
    -0.019946355,
    0.05248107,
    -0.062233597,
    0.015092951,
    0.009568825,
    -0.013843593,
    -0.053562496,
    -0.061670925,
    -0.035305984,
    0.032908812,
    0.04857493,
    0.034953453,
    0.023726141,
    0.03568444,
    0.0071175913,
    0.033559702,
    -0.0121424105,
    0.03407673,
    0.057266288,
    -0.012657767,
    0.06768064,
    -0.022355225,
    -0.030181471,
    0.12063107,
    0.012808932,
    0.030716958,
    -0.036874283,
    -0.008254023,
    -0.01081548,
    0.020372739,
    -0.027203616,
    0.0015654373,
    -0.039488234,
    -0.022952735,
    -0.0011268638,
    -0.009785246,
    -0.0019360933,
    0.016225388,
    -0.018154368,
    -0.014471327,
    -0.05883742,
    0.012274111,
    0.06887592,
    0.052230723,
    -0.06945254,
    -0.036823668,
    -0.012421808,
    0.035832312,
    0.02932272,
    -0.013966943,
    0.057543058,
    -0.023411168,
    0.025161343,
    -0.0649318,
    -0.06453301,
    -0.024157293,
    -0.03283516,
    0.0070455656,
    -0.024008652,
    -0.022920165,
    0.015138752,
    0.010144105,
    -0.008382338,
    0.044042453,
    0.018641355,
    -0.00897114,
    -0.022986831,
    -0.024504658,
    0.005061791,
    -0.046147745,
    0.017877372,
    0.055260062,
    -0.03701096,
    0.075398736,
    0.008653546,
    -0.021892056,
    -0.020546617,
    0.014034242,
    -0.072182335,
    0.013067999,
    -0.100795396,
    0.010451056,
    -0.07050469,
    -0.011445101,
    -0.03754845,
    0.006183421,
    -0.03906792,
    -0.0055965204,
    0.05707792,
    -0.010429032,
    -0.0011429724,
    0.005118022,
    -0.016644083,
    -0.04220137,
    -0.07363171,
    0.0024569791,
    -0.036916792,
    0.03395262,
    -0.02537861,
    0.07667082,
    0.035887685,
    0.015899615,
    -0.07066271,
    0.0015284163,
    0.044092063,
    -0.00084555487,
    -0.031053497,
    -0.009409518,
    0.019150222,
    0.039110914,
    -0.026089815,
    0.025691986,
    -0.052151006,
    0.072509676,
    -0.024798574,
    -0.03921443,
    0.008295112,
    -0.00042773844,
    -0.041100286,
    0.030542733,
    0.039614253,
    0.01804182,
    -0.0019106811,
    -0.0048501906,
    -0.03208538,
    -0.0050331447,
    0.026386712,
    0.024575505,
    0.05588996,
    0.030057926,
    0.01409509,
    -0.031694356,
    0.017517397,
    0.0046301717,
    0.007145245,
    0.07115179,
    0.0035148985,
    -0.014188136,
    0.007305263,
    -0.0128924055,
    0.0012173483,
    -0.010264721,
    -0.013301308,
    -0.021946715,
    0.012608889,
    0.06063012,
    -0.02197872,
    0.011334535,
    -0.017498214,
    0.040217794,
    -0.029707987,
    0.03888635,
    0.007113082,
    -0.10245564,
    0.019598834,
    0.042800713,
    -0.068540476,
    -0.007941252,
    0.029602854,
    0.012320459,
    0.04070456,
    0.0048046396,
    0.057080273,
    -0.041693073,
    0.0165682,
    -0.037762154,
    0.030711599,
    -0.023082186,
    0.06444822,
    0.02103803,
    -0.0064671254,
    0.035683487,
    -0.04167311,
    0.0337175,
    -0.020304035,
    -0.043164253,
    0.033318438,
    0.051226508,
    -0.09993186,
    0.023990132,
    0.035067804,
    -0.02158245,
    0.02973254,
    0.045522075,
    -0.037720405,
    -0.016477076,
    0.017233508,
    -0.031183288,
    -0.002931218,
    -0.006327033,
    -0.018274536,
    -0.0043489537,
    -0.010508976,
    0.010018209,
    0.029612847,
    0.092786245,
    0.030959306,
    -0.006647051,
    -0.027481854,
    0.033632692,
    -0.023163252,
    -0.010413391,
    0.04556712,
    -0.0027589782,
    0.029712437,
    0.06004067,
    -0.030746799,
    -0.03539816,
    0.02215609,
    -0.008097731,
    0.0035610283,
    0.060080826,
    -0.0088831335,
    -0.019353531,
    0.012506012,
    -0.04578203,
    0.026642656,
    0.034759957,
    0.03317829,
    0.06778447,
    -0.0039658644,
    -0.055566087,
    0.023027683,
    -0.041180346,
    -0.035393525,
    -0.027815914,
    0.014628632,
    -0.040841658,
    -0.02875485,
    0.017050356,
    -0.018104397,
    0.032889456,
    -0.052504424,
    0.05182534,
    -0.024680488,
    0.05011892,
    -0.030946976,
    0.012457738,
    -0.028428216,
    0.044254918,
    0.026299495,
    0.041920617,
    -0.01592546,
    -0.026756888,
    -0.021019576,
    -0.031648703,
    -0.008724201,
    0.0025635513,
    0.0113416435,
    -0.04281193,
    -0.042036083,
    -0.0067297197,
    -0.043461386,
    0.00083167566,
    -0.009508976,
    0.059358858,
    -0.014656265,
    -0.00035070922,
    -0.0049913535,
    0.034104995,
    0.044627994,
    0.04142639,
    0.038221695,
    0.024629146,
    -0.010383416,
    -0.008619005,
    0.029898448,
    -0.040645078,
    -0.055725325,
    0.029977558,
    -0.018410092,
    -0.11042691,
    0.012425915,
    0.040628515,
    -0.014945932,
    -0.021799535,
    0.041060366,
    0.028451148,
    -0.046030525,
    -0.017889444,
    -0.017596401,
    -0.016248139,
    -0.013585491,
    0.012225498,
    0.022918489,
    -0.0130026005,
    -0.036756888,
    -0.015256263,
    -0.06046553,
    0.034536958,
    0.010706609,
    -0.01574817,
    0.054597396,
    0.010708762,
    0.02091631,
    0.007813075,
    0.035738565,
    0.0096743,
    -0.05741579,
    -0.034075104,
    0.06407018,
    -0.058274616,
    0.040782902,
    0.019084051,
    -0.028117944,
    0.097568005,
    0.06757993,
    -0.04616895,
    0.01835175,
    0.009599453,
    0.009899384,
    -0.006697724,
    -0.030587083,
    -0.0057633854,
    -0.0035771965,
    0.005236175,
    0.03161966,
    -0.0014421007,
    0.02654683,
    -0.06784111,
    -0.033109434,
    0.0015561683,
    -0.028957006,
    -0.010133545,
    -0.009049643,
    0.095160015,
    0.038473047,
    0.015532988,
    -0.026837075,
    0.004548446,
    0.036702897,
    0.0034548303,
    -0.04240495,
    0.0085659055,
    0.03814674,
    -0.02157155,
    0.03191628,
    0.0018405454,
    -0.004841512,
    0.016154964,
    -0.022462817,
    0.04318009,
    -0.055848856,
    -0.037207298,
    0.05138956,
    0.047032602,
    -0.02519051,
    0.021451518,
    0.004464725,
    -0.055046022,
    0.02026468,
    0.056096263,
    -0.031548444,
    0.03434432,
    -0.048271492,
    0.0059668403,
    0.07247701,
    -0.01061879,
    0.054317787,
    -0.026415752,
    -0.024185874,
    0.08290211,
    -0.029081488,
    -0.03949319,
    0.055290263,
    -0.030485872,
    0.06952478,
    0.011051205,
    0.032816757,
    -0.012162714,
    -0.03957713,
    -0.04050204,
    0.027956164,
    0.016105121,
    0.0035087462,
    0.04936729,
    -0.04183739,
    0.0054607657,
    0.00022244385,
    -0.048364908,
    -0.05708388,
    -0.05690664,
    0.02973829,
    -0.0031154295,
    0.025180794,
    0.05131079,
    -0.052830625,
    -0.022286385,
    0.0075936974,
    0.016310522,
    0.027947418,
    -0.04063135,
    0.0024504068,
    0.011263393,
    0.021755524,
    -0.018598292,
    0.038954616,
    -0.0022197817,
    0.0030547322
  ]
},
{
  "_id": "54636",
  "content": "Employee number 54636, name  Jane Smith, department  Finance, location  Chicago, salary  130000",
  "embedding": [
    0.033282682,
    -0.03909191,
    0.0007547619,
    -0.00166024,
    0.039782085,
    0.0022312212,
    0.050308637,
    0.0008865104,
    0.0044103186,
    0.03072421,
    -0.0051613366,
    -0.026194286,
    -0.013544969,
    -0.011848663,
    -0.047196925,
    -0.0067790328,
    0.027002288,
    -0.020260727,
    -0.0076200166,
    -0.03599207,
    -0.035114743,
    -0.023491457,
    -0.0077283946,
    -0.026173472,
    0.021463424,
    0.0039335107,
    0.018358603,
    -0.106973566,
    -0.042405866,
    -0.018040666,
    -0.06100321,
    0.017380144,
    -0.02887869,
    0.04157278,
    -0.026515594,
    0.002928644,
    -0.012694755,
    0.012007126,
    0.0037733233,
    0.046051748,
    0.005322323,
    0.0059584975,
    -0.009696193,
    -0.0066717616,
    0.0436459,
    -0.022382155,
    0.032507967,
    -0.0065392246,
    0.025054803,
    -0.08322342,
    0.046882447,
    -0.024875944,
    0.06102459,
    -0.008675405,
    -0.0209155,
    -0.07849695,
    0.0020140782,
    -0.0011147283,
    -0.019611986,
    -0.025552176,
    -0.027990913,
    0.0037930773,
    -0.009247857,
    0.052749302,
    -0.0102324905,
    -0.062436964,
    -0.02680414,
    0.0022140676,
    0.06792232,
    0.003922225,
    -0.032647397,
    -0.03346009,
    0.07331381,
    -0.019998021,
    -0.06043405,
    -0.07097224,
    -0.018178217,
    0.007255711,
    0.05574352,
    0.03521836,
    -0.008382421,
    -0.0054113218,
    -0.038427114,
    -0.038553275,
    -0.03860179,
    0.06459174,
    0.029251965,
    -0.009218864,
    -0.01445029,
    0.046301886,
    -0.010660837,
    0.030970545,
    0.02029129,
    -0.07135827,
    -0.013141149,
    0.052297823,
    -0.014865988,
    -0.015496633,
    -0.016228134,
    -0.021733683,
    0.019089619,
    -0.016229896,
    -0.052840963,
    -0.008882637,
    -0.006283083,
    0.0034171198,
    -0.009800554,
    -0.00035290318,
    -0.025829503,
    0.02211951,
    -0.071032286,
    -0.021045374,
    -0.025188517,
    -0.048652653,
    0.07619293,
    -0.010589923,
    -0.059194453,
    0.053808484,
    0.041753646,
    0.0038279262,
    0.027524192,
    0.059054792,
    -0.0120413685,
    0.008316597,
    0.031715844,
    0.03668256,
    -0.01620795,
    0.00094534125,
    0.1083495,
    0.030639175,
    -0.029737566,
    -0.057788074,
    0.027845988,
    -0.009652855,
    0.055340156,
    0.0663029,
    0.04380604,
    -0.011622515,
    0.06322969,
    -0.019971412,
    0.003166246,
    0.022834063,
    -0.018897917,
    -0.0016617388,
    -0.02413109,
    0.0017157173,
    0.010872734,
    -0.011352978,
    0.006001529,
    0.015488254,
    0.0011707869,
    0.018522937,
    -0.018514102,
    0.02900784,
    0.020825073,
    0.022771334,
    -0.04606071,
    0.030873852,
    0.032193057,
    -0.010461426,
    0.062585354,
    0.015402362,
    -0.013989444,
    0.023873847,
    -0.0001557276,
    0.028785564,
    0.082039945,
    -0.04562552,
    -0.040299766,
    -0.027677026,
    0.018488018,
    0.041104753,
    -0.03979478,
    -0.030792084,
    -0.043800704,
    -0.07498492,
    -0.009120495,
    0.009481535,
    -0.006981507,
    -0.019725543,
    -0.033017475,
    -0.011190301,
    0.008700168,
    0.030437404,
    0.0058765025,
    0.03509147,
    0.04839477,
    -0.04233951,
    -0.0050839223,
    -0.00681981,
    0.03836845,
    0.004685588,
    -0.03892133,
    0.006600561,
    -0.0019655793,
    0.039389588,
    -0.01809466,
    0.011489714,
    -0.025835218,
    -0.018884161,
    -0.03204627,
    0.048295826,
    0.019568602,
    0.0053548636,
    0.033916872,
    -0.015454683,
    0.05538897,
    -0.033823296,
    -0.0039371606,
    0.0037781983,
    -0.0023951076,
    0.031877134,
    -0.028111735,
    -0.015856108,
    0.038164444,
    0.012281067,
    0.011310771,
    0.004349233,
    -0.014538568,
    -0.029191358,
    -0.006300531,
    0.019091565,
    -0.056574304,
    0.06351933,
    0.0007920405,
    -0.00020863887,
    -0.01369187,
    0.014471601,
    0.0048121824,
    -0.06280206,
    -0.0026793373,
    0.067380674,
    0.007164001,
    -0.038997144,
    0.054033946,
    -0.0022533264,
    0.013708685,
    0.038419407,
    -0.015276563,
    -0.004239497,
    -0.026898738,
    0.005635743,
    0.03847782,
    0.022422716,
    -0.030474218,
    0.0047512124,
    -0.04953256,
    0.015068766,
    0.020016206,
    0.042567335,
    0.0028769623,
    -0.03851735,
    0.046420325,
    0.011925669,
    -0.022733988,
    0.026839085,
    -0.07413135,
    -0.0020509947,
    0.012577789,
    -0.01844622,
    0.05583869,
    0.051888652,
    -0.0048309774,
    0.009347729,
    -0.049620014,
    0.047792498,
    -0.01790054,
    -0.0583071,
    -0.021284584,
    0.06689755,
    0.004781726,
    -0.012770478,
    0.07697373,
    0.03392616,
    0.014641257,
    0.008315434,
    0.004641932,
    0.057688177,
    -0.00011591461,
    -0.05162363,
    0.0132949175,
    0.054461304,
    0.04132465,
    -0.05228104,
    -0.008330981,
    0.024090331,
    0.0035575673,
    0.020221887,
    0.0046284744,
    -0.08109927,
    -0.08293294,
    -0.0143710505,
    -0.00088063197,
    -0.066603884,
    -0.05854246,
    0.023163913,
    -0.048908163,
    0.016917551,
    0.0307055,
    0.006515332,
    0.0063499426,
    -0.03967575,
    -0.04001876,
    -0.05907809,
    0.03273935,
    0.01719176,
    0.02424203,
    -0.072151884,
    -0.016568365,
    -0.013442307,
    0.0270092,
    -0.034306023,
    -0.03671066,
    -0.0067472355,
    0.019714633,
    0.057815082,
    -0.012577134,
    -0.013382564,
    -0.03911529,
    0.050835066,
    -0.0037874833,
    -0.008278123,
    0.042883575,
    -0.052752126,
    -0.0025583908,
    0.033462517,
    -0.00065002317,
    0.04533471,
    -0.0154035175,
    0.031390563,
    -0.0136398645,
    0.05136288,
    -0.005182816,
    0.063051686,
    0.0050854594,
    0.035023358,
    -0.083840184,
    0.006783878,
    -0.08204774,
    0.007304815,
    0.063131176,
    0.007564634,
    -0.054168995,
    -0.04331101,
    0.011824145,
    -0.03706767,
    -0.013556429,
    0.040878236,
    0.0807747,
    0.055227622,
    -0.018239543,
    0.012912111,
    -0.022989053,
    -0.0028701182,
    -0.031126976,
    -0.0047545466,
    0.029136088,
    -0.017281001,
    0.01922514,
    -0.012010304,
    -0.04137593,
    0.034444544,
    -0.02019,
    0.019743202,
    0.011670469,
    -0.007273119,
    0.013197889,
    0.028474236,
    -0.01086087,
    0.021434411,
    0.024562098,
    -0.04574378,
    0.0064201616,
    0.0066477978,
    -0.011263019,
    -0.053637076,
    -0.038792353,
    -0.027472662,
    0.028990641,
    0.048245337,
    0.033630706,
    0.0068463734,
    -0.008434694,
    0.05721538,
    0.032129478,
    0.00040128073,
    0.012669716,
    0.05994472,
    -0.0030158863,
    0.0371618,
    -0.0053152503,
    -0.016319215,
    0.11017134,
    -0.0046604937,
    0.0047023036,
    -0.010484685,
    -0.011737418,
    -0.029090436,
    0.017523576,
    -0.03561854,
    -0.015012307,
    -0.043565813,
    -0.0042169513,
    -0.015257499,
    -0.0013578214,
    -0.020777592,
    -0.0026453973,
    -0.037206873,
    0.01621471,
    -0.033481725,
    0.023214078,
    0.07240724,
    0.038964514,
    -0.10446012,
    -0.024342356,
    -0.012709968,
    0.06211316,
    0.028951973,
    -0.017876118,
    0.033162788,
    -0.01688093,
    -0.0017007018,
    -0.06386243,
    -0.06569357,
    -0.019704562,
    0.0058997083,
    0.02682609,
    0.0019023331,
    0.0017976766,
    0.047627747,
    -0.0009570554,
    0.003511069,
    -0.028590811,
    0.008931729,
    0.0056299744,
    -0.045468964,
    -0.015844844,
    0.024054134,
    -0.027469898,
    0.022173349,
    0.08148612,
    -0.019955546,
    0.071963444,
    0.017910395,
    -0.03124338,
    -0.025000984,
    0.0054683266,
    -0.043931495,
    0.0008234161,
    -0.088591345,
    0.000479956,
    -0.060126673,
    -0.03357947,
    -0.056410566,
    -0.032589775,
    -0.061390713,
    0.023527699,
    0.06446886,
    -0.007932508,
    -0.0031587793,
    -0.038047824,
    -0.016549844,
    -0.020477492,
    -0.075683415,
    -0.012110268,
    -0.04891436,
    0.026568849,
    -0.029075945,
    0.088291675,
    0.05912708,
    0.015805336,
    -0.049823914,
    0.024532974,
    0.045406923,
    0.0095926905,
    -0.020348761,
    -0.027956523,
    0.030030653,
    0.04987238,
    -0.032793604,
    -0.011533045,
    -0.023860402,
    0.062345207,
    -0.019582005,
    -0.031335227,
    0.04970386,
    -0.023734426,
    -0.021209128,
    0.026652478,
    0.030278446,
    0.0072057345,
    0.015214752,
    0.030629016,
    -0.038683932,
    -0.019320292,
    -0.002425624,
    0.027636893,
    0.061737496,
    0.024350017,
    0.056657396,
    -0.026632559,
    0.031648163,
    0.017482013,
    -0.0008389236,
    0.059907097,
    -0.02123141,
    -0.0025512732,
    0.0016537616,
    0.013569981,
    0.0036193815,
    0.0020932176,
    -0.024130942,
    0.0026975519,
    -0.0041695293,
    0.04122382,
    -0.011495905,
    -0.008079376,
    -0.0057473094,
    0.039514784,
    -0.04971112,
    0.038069297,
    -0.026460811,
    -0.09403351,
    0.0051518404,
    0.041077457,
    -0.02429841,
    0.030635186,
    0.023150187,
    -0.015075604,
    0.018495653,
    -0.015820375,
    0.06806306,
    -0.041917916,
    -0.0020552087,
    -0.03989304,
    0.0018853423,
    -0.03138397,
    0.043138567,
    0.03781449,
    -0.022061005,
    0.05022614,
    -0.024115918,
    0.060577173,
    -0.015132834,
    -0.036388364,
    -0.008102943,
    0.04300946,
    -0.06315613,
    0.0036877454,
    0.01845251,
    -0.01958063,
    0.020833353,
    0.04074978,
    -0.016953025,
    -0.022429537,
    0.014793756,
    -0.040389486,
    -0.013773572,
    -0.009218371,
    0.005113001,
    -0.05334978,
    -0.017054679,
    0.0047935587,
    0.008970996,
    0.090954326,
    0.040972985,
    -0.012958875,
    -0.048973564,
    0.036145844,
    -0.014689881,
    -0.019061541,
    0.011643549,
    0.0013238543,
    0.010562913,
    0.05699919,
    -0.033291608,
    -0.038737576,
    0.033350945,
    0.004073598,
    -0.0031493988,
    0.064587645,
    0.0049619107,
    -0.033150792,
    0.009891267,
    -0.05739713,
    0.037735194,
    0.062099792,
    -0.0033623695,
    0.07860333,
    0.009854467,
    -0.06447139,
    0.031053409,
    -0.058900006,
    -0.05029678,
    -0.027897462,
    0.056934282,
    -0.023755403,
    -0.02699664,
    0.0148529,
    -0.01666892,
    -0.0018880437,
    -0.0457452,
    0.053869832,
    -0.017661236,
    0.041393574,
    -0.029723926,
    0.035513297,
    -0.033740062,
    0.017888788,
    0.025985245,
    0.04350399,
    -0.005232885,
    -0.0005693812,
    -0.0033883732,
    -0.037065566,
    -0.02527872,
    -0.0013861161,
    -0.009989347,
    0.017875295,
    -0.024106601,
    0.019750569,
    -0.010035022,
    0.023742517,
    -0.011065428,
    0.033420607,
    0.010958177,
    -0.01141841,
    0.004739421,
    0.09271151,
    0.058798864,
    0.052398294,
    0.024582228,
    0.031933635,
    -0.005510293,
    -0.055356447,
    0.04904649,
    -0.016862152,
    -0.057705753,
    0.024840772,
    -0.00048015165,
    -0.10546246,
    0.033585515,
    0.054793786,
    -0.0070182765,
    -0.031317633,
    0.034208145,
    0.015181784,
    -0.062814064,
    -0.019155085,
    -0.03985083,
    -0.0043001687,
    -0.027612321,
    0.025576307,
    0.020972272,
    -0.015960751,
    -0.03785616,
    -0.03439213,
    -0.035648625,
    0.03173273,
    0.030685507,
    -0.018922847,
    0.067803204,
    -0.0148015395,
    0.044523258,
    0.0061558536,
    0.0054381737,
    0.016035475,
    -0.07147066,
    -0.04419202,
    0.07741728,
    -0.045105446,
    0.03828778,
    0.0073140706,
    -0.05118925,
    0.061084356,
    0.05001082,
    -0.041532677,
    -0.0016793367,
    0.019244568,
    0.005559429,
    0.017637422,
    -0.012980126,
    0.013700538,
    0.04216373,
    0.026694974,
    0.043757316,
    0.012165836,
    0.023159562,
    -0.056082647,
    -0.030905947,
    0.016382935,
    -0.0074324473,
    -0.016718535,
    -0.0066933725,
    0.07567363,
    -0.009259794,
    0.019372217,
    0.014743861,
    0.020617943,
    0.057029292,
    0.0050983094,
    -0.032250557,
    0.013374774,
    0.047974505,
    -0.049952496,
    0.012993495,
    0.02056461,
    -0.0057155536,
    0.021704368,
    -0.0010753982,
    0.03720598,
    -0.062292766,
    -0.03158331,
    0.04137522,
    0.013299325,
    0.0049977377,
    0.04511127,
    -0.013574835,
    -0.033320617,
    0.0153828645,
    0.05682976,
    -0.015472821,
    -0.0013420929,
    -0.054665994,
    -0.0047995877,
    0.056595206,
    -0.03133152,
    0.08055057,
    -0.019866763,
    -0.008866304,
    0.10177134,
    -0.035269864,
    -0.027235541,
    0.04055463,
    -0.0308909,
    0.08752392,
    0.036332566,
    0.022787439,
    -0.036100462,
    -0.08477476,
    -0.059534717,
    0.03207563,
    0.013661438,
    0.013927431,
    0.025811398,
    -0.025891041,
    0.030257259,
    -0.020082533,
    -0.010865357,
    -0.07682985,
    -0.08710289,
    0.026793221,
    -0.03599497,
    0.036737897,
    0.038387842,
    -0.067557946,
    -0.018947005,
    -0.020812513,
    0.04308112,
    0.0135874795,
    -0.026089147,
    -0.00018917858,
    -0.033624545,
    0.038969826,
    0.027176073,
    0.034541093,
    -0.0001338307,
    0.009559905
  ]
},
{
  "_id": "54635",
  "content": "Employee number 54635, name  John Smith, department  Engineering, location  San Francisco, salary  150000",
  "embedding": [
    0.02751842,
    -0.035257466,
    -0.017211914,
    -0.00021509113,
    0.065513395,
    0.015236125,
    0.046573777,
    -0.027342448,
    0.013279674,
    0.015880402,
    -0.0026871182,
    -0.020755854,
    -0.028976398,
    -0.025017332,
    -0.02915205,
    0.03748723,
    0.026784075,
    -0.008561052,
    0.0061514685,
    -0.08216181,
    -0.012679269,
    -0.019104697,
    -0.035561316,
    -0.013733142,
    -0.002272582,
    -0.014719888,
    0.030248245,
    -0.08619838,
    -0.03771919,
    0.018865846,
    -0.063968174,
    0.03219376,
    -0.035308257,
    0.036639687,
    -0.04739701,
    0.01655839,
    -0.025951466,
    0.019777248,
    0.008441114,
    0.0397848,
    -0.007323799,
    -0.010856907,
    -0.015435451,
    -0.024571002,
    0.0608382,
    -0.001076124,
    -0.01160508,
    0.027767012,
    0.020741891,
    -0.08319567,
    0.04286476,
    -0.0065871845,
    0.030671211,
    -0.008211031,
    -0.05312356,
    -0.07651591,
    -0.013340908,
    -0.012418467,
    -0.02554261,
    -0.027668754,
    -0.028723458,
    0.0072301184,
    0.018877236,
    0.03554761,
    -0.004688246,
    -0.07433135,
    -0.0035417427,
    -0.006659041,
    0.04192288,
    -0.0207361,
    -0.006254261,
    -0.024812873,
    0.059861075,
    -0.03134214,
    -0.07463872,
    -0.11254215,
    -0.007414231,
    0.01804921,
    0.057197362,
    0.02665551,
    0.0064427084,
    -0.013575005,
    -0.029392047,
    -0.012261425,
    -0.030408913,
    0.074111775,
    0.016163299,
    -0.017348083,
    -0.027919967,
    0.040365145,
    0.016613623,
    0.0008838358,
    0.035404228,
    -0.07160502,
    -0.014951479,
    0.06784549,
    0.0066313925,
    -0.052722607,
    -0.029260378,
    -0.053105693,
    -0.0045598387,
    -0.05004554,
    -0.052556243,
    -0.0067943437,
    0.019090641,
    0.007432553,
    -0.020073954,
    -0.0042186803,
    -0.005367988,
    0.018064711,
    -0.073992364,
    -0.017997328,
    -0.032181244,
    -0.04996697,
    0.06846196,
    -0.03522803,
    -0.035346393,
    0.07071394,
    0.041346475,
    0.007324973,
    0.08921032,
    0.068063006,
    0.020346623,
    0.040123172,
    -0.010046104,
    0.025201118,
    -0.003961309,
    0.023122495,
    0.09587856,
    0.05031979,
    -0.01057047,
    -0.04394514,
    0.034751166,
    -0.014335325,
    0.03341888,
    0.05706479,
    0.05054945,
    0.004166189,
    0.061515614,
    -0.022616953,
    -0.0050968137,
    0.0010603444,
    -0.030782396,
    0.008184928,
    -0.011474374,
    0.003339862,
    -0.0067458292,
    -0.010549199,
    0.027625475,
    -0.0020265754,
    0.006763103,
    0.009542199,
    -0.03164191,
    0.027494777,
    0.015781684,
    0.010094312,
    -0.026690045,
    0.027364649,
    0.020415021,
    -0.0379163,
    0.032620024,
    0.040363166,
    -0.005355615,
    0.02923704,
    0.0073294668,
    0.008268878,
    0.046909377,
    -0.035831597,
    -0.037963904,
    -0.05067842,
    -0.023855058,
    0.031068098,
    -0.060321517,
    -0.017361106,
    -0.03714339,
    -0.052008033,
    -0.017402591,
    -0.0000853109,
    -0.01522777,
    0.004505938,
    -0.035380103,
    -0.04122089,
    0.017592149,
    0.017477088,
    0.009680622,
    0.0601484,
    0.053315826,
    -0.04279214,
    0.009589118,
    -0.010029888,
    0.020045798,
    0.024653148,
    -0.023524566,
    0.011961308,
    0.0017022899,
    0.029609565,
    0.028629072,
    0.021015882,
    0.00012135336,
    -0.04946617,
    -0.034740772,
    0.031152897,
    0.027573282,
    -0.017018853,
    0.020141952,
    -0.020575022,
    0.08574167,
    -0.025361745,
    0.0000718993,
    0.07202345,
    -0.035066936,
    0.042211156,
    -0.02737654,
    -0.007903302,
    0.055386215,
    0.03386666,
    0.007098444,
    -0.0011916764,
    -0.0077267145,
    -0.039679147,
    0.0109159555,
    0.013889261,
    -0.020267498,
    0.057652816,
    -0.01675666,
    -0.01046991,
    0.0063280216,
    0.036489647,
    0.014317979,
    -0.04359035,
    -0.0038247926,
    0.03433474,
    0.0046200114,
    -0.028734986,
    0.033694655,
    -0.0072179954,
    0.0065697636,
    0.0035546091,
    -0.018106835,
    0.0008928709,
    -0.04377693,
    0.015936524,
    0.026221775,
    0.004182328,
    -0.055179127,
    0.010751439,
    -0.038998943,
    0.039809167,
    -0.015138335,
    0.009765969,
    -0.0081632165,
    -0.05567452,
    0.05554104,
    0.026053185,
    -0.05740401,
    0.03465235,
    -0.076469,
    0.0015592162,
    0.009639975,
    -0.030025609,
    0.05954854,
    0.021440376,
    -0.006658576,
    0.02273896,
    -0.031426214,
    0.040050857,
    -0.021161372,
    -0.068565525,
    -0.028184833,
    0.068864435,
    -0.005223995,
    -0.006345874,
    0.07794489,
    0.046531476,
    0.035680976,
    -0.010769412,
    -0.006249256,
    0.041365553,
    0.003833638,
    -0.058614638,
    -0.0052710325,
    0.07343842,
    0.03782047,
    -0.046215393,
    -0.029175224,
    0.011108449,
    -0.0022866763,
    0.010857836,
    0.008042948,
    -0.07578336,
    -0.09506901,
    0.0028703813,
    -0.017622843,
    -0.05017095,
    -0.06601734,
    0.016744547,
    -0.030435143,
    0.010448069,
    0.023559375,
    0.003017711,
    -0.010440744,
    -0.051503733,
    -0.031850483,
    -0.06224817,
    0.03302146,
    0.0022398247,
    0.012647616,
    -0.05779213,
    -0.012409115,
    -0.01943723,
    0.027941398,
    -0.022145638,
    -0.015692553,
    -0.0011641445,
    0.026432425,
    0.05613625,
    0.010688817,
    -0.014724717,
    -0.029040305,
    0.07541772,
    -0.0038578296,
    0.0063860323,
    0.028974347,
    -0.048377227,
    -0.0020083082,
    0.061093163,
    -0.0015345091,
    0.04327932,
    -0.006280553,
    -0.004461047,
    -0.004969216,
    0.079298794,
    0.005280641,
    0.075993665,
    -0.00903695,
    0.01402909,
    -0.088527754,
    0.007002133,
    -0.06112628,
    -0.025775367,
    0.035340235,
    0.011317101,
    -0.058157604,
    -0.057730775,
    -0.030742848,
    -0.024909437,
    -0.01292577,
    0.025219142,
    0.07330923,
    0.04195776,
    0.0330579,
    0.027340477,
    -0.018308474,
    -0.040544774,
    -0.043364618,
    -0.017788624,
    0.024081899,
    -0.02448898,
    0.025725441,
    -0.014032703,
    -0.03716917,
    0.030908864,
    0.0046337834,
    -0.017839137,
    0.025155617,
    -0.005961568,
    0.0017926248,
    0.010721402,
    0.008076512,
    0.000014950954,
    0.03687499,
    -0.080296755,
    -0.0022581385,
    0.0135227805,
    -0.007616169,
    -0.059810784,
    -0.06190626,
    -0.022344204,
    0.04095455,
    0.024678344,
    0.040884405,
    0.01146623,
    -0.010222091,
    0.023510607,
    0.013003448,
    -0.024498258,
    0.01566375,
    0.03966229,
    -0.013284801,
    0.05625292,
    0.0035774496,
    -0.041765593,
    0.075709105,
    0.0025806301,
    0.002827293,
    -0.0127510615,
    0.0076297605,
    -0.018482147,
    0.0195352,
    -0.008572333,
    -0.023895975,
    -0.022642754,
    -0.021384051,
    -0.025655594,
    0.011634965,
    -0.040602997,
    -0.0025013296,
    -0.019177016,
    0.025949784,
    -0.044972513,
    0.036257338,
    0.07511653,
    0.036459416,
    -0.115887314,
    -0.03673168,
    -0.027124275,
    0.06939686,
    0.022735972,
    -0.017979583,
    0.03082303,
    -0.017964998,
    -0.002043333,
    -0.045927435,
    -0.083149225,
    -0.023609241,
    0.008159602,
    -0.0014957677,
    -0.019364858,
    0.017326526,
    0.03703413,
    0.021754017,
    -0.024310483,
    0.012974437,
    0.0025437805,
    0.009190147,
    -0.03622636,
    -0.025097648,
    0.03175117,
    -0.060564324,
    0.025262883,
    0.045786444,
    -0.03756287,
    0.077972464,
    0.014882087,
    -0.035983026,
    -0.03335668,
    0.012810578,
    -0.05117928,
    0.016779415,
    -0.09439797,
    -0.0063598235,
    -0.046981793,
    -0.0028302062,
    -0.058332726,
    -0.025126418,
    -0.0774988,
    -0.005797502,
    0.051930707,
    -0.031145284,
    -0.017120061,
    -0.019712316,
    -0.03858973,
    -0.0066188793,
    -0.07773236,
    0.014307085,
    -0.027230764,
    0.0331902,
    -0.057089612,
    0.057789687,
    0.06540567,
    0.022184646,
    -0.048069406,
    0.015343113,
    0.0558476,
    0.0001679844,
    -0.019582808,
    -0.026129218,
    0.030448413,
    0.03634036,
    -0.041653648,
    -0.03448554,
    -0.023397086,
    0.076667525,
    -0.009382471,
    -0.03721751,
    0.023369618,
    -0.014296145,
    -0.008192296,
    -0.009530903,
    0.03565617,
    0.017519575,
    0.02974625,
    0.003261011,
    -0.048980772,
    -0.011459121,
    -0.020589355,
    0.00025388523,
    0.05537085,
    0.03211041,
    0.020657161,
    -0.008744432,
    0.022239113,
    0.03172477,
    -0.012103961,
    0.06935857,
    -0.030093772,
    -0.022945607,
    0.006408147,
    0.020852357,
    0.028491085,
    -0.0023250037,
    -0.002611426,
    -0.017636396,
    -0.02044602,
    0.03541466,
    -0.016902495,
    -0.006867505,
    -0.00012285702,
    0.017117904,
    -0.020677648,
    0.0557191,
    -0.015488911,
    -0.10235103,
    -0.0028210408,
    0.010193204,
    -0.032251876,
    0.01060267,
    0.03557713,
    -0.0063216602,
    0.009259219,
    0.0051827505,
    0.038854543,
    -0.031460393,
    -0.0076761693,
    -0.026699234,
    0.018988753,
    -0.0137761375,
    0.046094075,
    0.016433887,
    -0.003973164,
    0.063081056,
    -0.029332547,
    0.054982897,
    -0.03574009,
    -0.053488664,
    -0.0065793497,
    0.04935732,
    -0.07006432,
    0.02812001,
    0.00647268,
    -0.0075788135,
    0.022446249,
    0.0481543,
    -0.028088065,
    -0.00019163998,
    0.0115054725,
    -0.045824103,
    -0.0041767946,
    0.0011554541,
    -0.010604986,
    -0.00820998,
    -0.015158567,
    0.006710291,
    0.027485032,
    0.0827062,
    0.035160348,
    -0.0056700693,
    -0.06100241,
    0.03775862,
    -0.055310618,
    0.0020003193,
    0.003878855,
    0.0039324365,
    0.021707052,
    0.056594085,
    -0.023516221,
    -0.051027056,
    0.02914387,
    -0.01747557,
    -0.00048578077,
    0.06910575,
    -0.02813352,
    -0.0067434846,
    0.02015622,
    -0.030215016,
    0.013908208,
    0.0559234,
    0.007264178,
    0.07277442,
    -0.0056195944,
    -0.07750021,
    0.050576977,
    -0.06140354,
    -0.045618854,
    -0.020566126,
    0.039101325,
    -0.025846323,
    -0.014014278,
    -0.010953099,
    0.0053575314,
    0.01853153,
    -0.059806224,
    0.079701826,
    -0.026566066,
    0.040181253,
    -0.02266395,
    0.0054217833,
    -0.027931219,
    0.04334095,
    0.03031099,
    0.02989241,
    -0.00061276584,
    -0.013604992,
    0.008013184,
    -0.03547994,
    -0.01926322,
    -0.007489133,
    -0.0029699102,
    -0.010580029,
    -0.018646544,
    0.0047476776,
    -0.020718446,
    0.009074871,
    -0.0027520477,
    0.0387986,
    -0.023973802,
    -0.01189377,
    0.013701782,
    0.068496615,
    0.04820877,
    0.019212667,
    0.045648925,
    0.0318747,
    -0.028331727,
    -0.025103705,
    0.0328582,
    -0.03484345,
    -0.05530036,
    0.011778919,
    -0.005664444,
    -0.113750234,
    0.018433394,
    0.03918017,
    -0.012234621,
    -0.016486265,
    0.017421383,
    0.024043733,
    -0.04742088,
    0.020483378,
    -0.038147032,
    -0.009845963,
    -0.006499819,
    0.028929604,
    0.025346762,
    -0.012089239,
    -0.032945115,
    -0.04614911,
    -0.028733844,
    0.03818017,
    0.04198492,
    -0.024049556,
    0.06125142,
    -0.008327446,
    0.030352222,
    0.015361837,
    -0.013292321,
    0.006475574,
    -0.072518654,
    -0.036767293,
    0.0740501,
    -0.06637531,
    0.043207835,
    0.034058686,
    -0.03576254,
    0.047720406,
    0.037699528,
    -0.023285469,
    0.025489798,
    0.002287648,
    -0.008411476,
    0.02260013,
    -0.020512147,
    0.008027023,
    0.029532177,
    0.0059477957,
    0.04624887,
    0.021156397,
    0.036551874,
    -0.041027997,
    -0.049307615,
    0.02526815,
    -0.02010938,
    -0.019960264,
    -0.014263981,
    0.079093084,
    0.010921492,
    0.018967591,
    -0.008221532,
    0.0058250814,
    0.07463721,
    -0.03572568,
    -0.013496767,
    -0.00042068915,
    0.019645795,
    -0.049485173,
    0.03608238,
    -0.01177695,
    0.0020465946,
    0.012326075,
    -0.0023621495,
    0.049434356,
    -0.078708716,
    -0.048812617,
    0.015036083,
    0.020805584,
    -0.0033854137,
    0.0066305967,
    0.015715208,
    -0.027279327,
    0.03890442,
    0.028212277,
    -0.025578374,
    -0.0042159823,
    -0.04713265,
    0.0012227457,
    0.07078375,
    -0.028477665,
    0.05951706,
    -0.0190399,
    -0.026970295,
    0.06499598,
    -0.034548096,
    -0.036340587,
    0.0422616,
    -0.03698566,
    0.0678964,
    0.012707417,
    0.03466419,
    -0.015625846,
    -0.08137007,
    -0.045645062,
    0.023715043,
    0.0023143874,
    0.008170114,
    0.041747324,
    -0.022417393,
    0.011471595,
    -0.020469397,
    -0.02913878,
    -0.07025473,
    -0.07867984,
    0.025465682,
    -0.026013091,
    -0.011840964,
    0.047041006,
    -0.058210913,
    -0.027502269,
    -0.01028805,
    0.029465359,
    0.03936156,
    -0.028816642,
    0.022757342,
    -0.002236254,
    0.035480563,
    0.0058484883,
    0.026348954,
    -0.016547782,
    0.018939447
  ]
},
{
  "_id": "54637",
  "content": "Employee number 54637, name  John Johnson, department  HR, location  Miami, salary  110000",
  "embedding": [
    0.0151990615,
    -0.018385835,
    -0.031483263,
    0.0082740635,
    0.047831737,
    0.022178285,
    0.042604085,
    -0.015300213,
    0.0052842684,
    0.04275969,
    -0.031947587,
    -0.055677623,
    -0.01569269,
    -0.016129037,
    -0.044204526,
    0.0340167,
    0.0040199882,
    -0.014162755,
    0.011139275,
    -0.018563714,
    -0.03279649,
    0.0037455678,
    -0.01916476,
    -0.02534246,
    0.020670477,
    -0.010130891,
    0.028611206,
    -0.084460564,
    -0.04089965,
    -0.006191601,
    -0.059610564,
    0.07042831,
    -0.0019582002,
    0.040124465,
    -0.010672403,
    0.00023288652,
    -0.036172427,
    0.019273408,
    0.022685157,
    0.019930484,
    -0.0069136596,
    -0.018724103,
    -0.027178712,
    0.0102139255,
    0.060994297,
    0.01205306,
    0.008931071,
    0.030500712,
    0.039762385,
    -0.0844163,
    0.03557818,
    0.016239291,
    0.011505173,
    0.016626962,
    -0.051115632,
    -0.089058846,
    0.006736814,
    0.0016996651,
    -0.018498152,
    -0.02900407,
    -0.037330467,
    -0.0019586603,
    -0.006318982,
    0.011514259,
    -0.020778127,
    -0.060733095,
    -0.03416104,
    0.003209011,
    0.037856832,
    -0.02291265,
    -0.02566606,
    -0.04075286,
    0.06387488,
    -0.013900549,
    -0.06662302,
    -0.11717324,
    -0.021233523,
    0.03737273,
    0.062059958,
    0.018934567,
    -0.0021977667,
    -0.017087216,
    -0.03001248,
    -0.029826604,
    -0.053601734,
    0.07904818,
    -0.0050950386,
    -0.027237581,
    -0.009588286,
    0.02798285,
    -0.015427634,
    0.010472741,
    -0.019344417,
    -0.0884872,
    -0.03772547,
    0.07433684,
    -0.005178444,
    -0.04721746,
    0.016155189,
    -0.020898396,
    0.027348634,
    -0.03220877,
    -0.039294083,
    0.025916431,
    0.03894846,
    0.0050957613,
    0.011876476,
    0.009665685,
    -0.0010955081,
    -0.0035323082,
    -0.056610696,
    0.0042955126,
    0.004623993,
    -0.047537014,
    0.065262586,
    -0.045530386,
    -0.043586448,
    0.08124031,
    0.0058494746,
    0.017003875,
    0.06947752,
    0.045389757,
    -0.00809288,
    0.003066964,
    0.024357231,
    0.05353458,
    0.0041651297,
    0.015048914,
    0.09423512,
    0.059014294,
    -0.01673688,
    -0.025130406,
    0.030101426,
    0.00313877,
    0.05056692,
    0.05799128,
    0.06417359,
    0.0070831957,
    0.061689373,
    -0.025019836,
    -0.013782963,
    -0.006486025,
    -0.04184629,
    0.0021818338,
    -0.0029972838,
    -0.0069409898,
    -0.008351021,
    -0.009658322,
    0.048463274,
    -0.029926352,
    0.024303105,
    -0.017629249,
    -0.014122519,
    0.006829436,
    0.015102068,
    0.02918675,
    -0.047867116,
    0.017428437,
    0.004246343,
    -0.02182751,
    0.05868468,
    -0.0012855188,
    -0.005099442,
    0.001516126,
    -0.002746483,
    -0.0103094075,
    0.009653553,
    -0.03586081,
    -0.042287398,
    -0.060810238,
    0.007971088,
    0.030602243,
    -0.07896841,
    -0.019366264,
    -0.059197318,
    -0.040017575,
    -0.0060959007,
    0.0067164223,
    -0.031493124,
    -0.024010602,
    -0.037667226,
    -0.03403944,
    -0.00097576715,
    0.027220456,
    -0.0021284383,
    0.06373505,
    0.041661903,
    -0.025651291,
    -0.0026189024,
    -0.026962018,
    0.04662318,
    0.029713636,
    -0.0072639524,
    -0.008331813,
    -0.013575179,
    0.030916931,
    0.015517671,
    0.028786736,
    0.0042063724,
    -0.029553477,
    -0.016450562,
    0.060844745,
    -0.0013221892,
    -0.0055684242,
    0.04769308,
    -0.038562633,
    0.04191216,
    -0.07439696,
    -0.013617095,
    0.04565873,
    0.0075634466,
    0.04673023,
    -0.041626494,
    -0.03232615,
    0.04359808,
    0.024681205,
    0.024057476,
    0.007089288,
    -0.00020997692,
    -0.05840243,
    0.0011099685,
    0.012605227,
    -0.020300457,
    0.072397396,
    0.0029569077,
    -0.012561521,
    -0.0031120302,
    0.04246921,
    0.019347874,
    -0.016595539,
    0.008932043,
    0.038021155,
    0.022360448,
    -0.041870937,
    0.03759141,
    0.017054925,
    0.023967758,
    0.017783063,
    -0.02351047,
    0.023144009,
    -0.054543506,
    0.012925695,
    0.08040064,
    0.007308367,
    -0.08529201,
    -0.0056034215,
    -0.028788855,
    0.00034720462,
    0.014641983,
    0.024667779,
    -0.028710028,
    -0.041735,
    0.08198758,
    0.041555718,
    -0.04926944,
    0.052032072,
    -0.086632214,
    -0.00159897,
    -0.009663495,
    -0.0071806083,
    0.051270913,
    0.024380185,
    -0.01310986,
    0.021249343,
    -0.032247756,
    0.040103268,
    -0.0109099755,
    -0.07212998,
    -0.018791035,
    0.047924884,
    -0.01295749,
    -0.00022769881,
    0.08965714,
    0.056537516,
    0.039999932,
    0.011153844,
    -0.0015653945,
    0.052498676,
    0.0031664725,
    -0.04293477,
    -0.018758398,
    0.045290086,
    0.025194753,
    -0.036377974,
    -0.013312391,
    -0.004944805,
    -0.0059067444,
    -0.019272402,
    0.011710215,
    -0.06544868,
    -0.08821586,
    0.017618323,
    -0.025412118,
    -0.053164277,
    -0.046637923,
    0.004189994,
    -0.029162928,
    0.016743293,
    0.017788872,
    0.02451719,
    0.011813669,
    -0.014297119,
    0.0047462014,
    -0.0604839,
    0.030824589,
    -0.011509641,
    0.030518167,
    -0.06083328,
    -0.008108362,
    -0.010405061,
    0.0279155,
    -0.030137705,
    -0.037425663,
    -0.003826426,
    0.045524806,
    0.04506571,
    -0.0003876464,
    -0.025874265,
    -0.035840876,
    0.04633308,
    0.015148351,
    0.02118069,
    0.022964032,
    -0.015708314,
    -0.012418845,
    0.017429173,
    -0.011633802,
    0.026676752,
    0.016578717,
    0.00702841,
    -0.030563941,
    0.028610306,
    0.014839663,
    0.079686195,
    -0.004785117,
    0.027001834,
    -0.06591138,
    0.008991921,
    -0.08665218,
    -0.024576634,
    0.042076416,
    -0.002764758,
    -0.037591983,
    -0.06974853,
    -0.008148083,
    -0.0010839726,
    -0.03883453,
    0.029932331,
    0.07715753,
    0.0428311,
    0.00048389743,
    0.02941479,
    -0.023215424,
    -0.021311667,
    -0.029051634,
    -0.0078097307,
    0.05971781,
    -0.0003501407,
    0.020407172,
    -0.020588633,
    -0.045638587,
    0.05452016,
    -0.033031806,
    0.0018599117,
    0.021753361,
    -0.017417207,
    0.011331878,
    0.025535421,
    -0.012881257,
    0.0020049305,
    0.027690342,
    -0.05566046,
    -0.009303709,
    0.02051795,
    -0.0012120871,
    -0.05989722,
    -0.05193341,
    -0.03752882,
    0.0151110515,
    0.04022004,
    0.059206907,
    -0.0004753494,
    -0.015862446,
    0.008765484,
    0.002500967,
    -0.024079999,
    0.037471678,
    0.04361219,
    -0.009226066,
    0.06009437,
    -0.0072968435,
    -0.03503233,
    0.12823524,
    -0.019953987,
    0.022029717,
    -0.03506259,
    -0.0069630756,
    -0.010650351,
    0.00024113746,
    -0.005909056,
    -0.026001915,
    -0.05297878,
    -0.019525815,
    -0.011212167,
    -0.0011330652,
    -0.029903706,
    -0.013018626,
    -0.04529402,
    0.022067638,
    -0.042428583,
    0.011411191,
    0.07941165,
    0.046710193,
    -0.09081757,
    -0.020232175,
    -0.021946874,
    0.04843305,
    0.025839098,
    -0.029347481,
    0.052311007,
    -0.019753845,
    0.015101981,
    -0.02324361,
    -0.06853173,
    -0.013895659,
    -0.027288755,
    0.014830132,
    -0.019731691,
    -0.012770851,
    0.047408056,
    0.010517912,
    -0.018772654,
    0.008655169,
    0.004405761,
    0.028954867,
    -0.043041132,
    -0.005176918,
    -0.01609051,
    -0.028849607,
    0.05003634,
    0.029869856,
    -0.050169658,
    0.08384437,
    0.01797906,
    -0.01214695,
    -0.027093818,
    0.025474763,
    -0.035368394,
    -0.0013650756,
    -0.10130171,
    -0.019761328,
    -0.058286637,
    -0.031827793,
    -0.018027933,
    -0.020658895,
    -0.06964426,
    0.015475856,
    0.06776502,
    -0.014362135,
    -0.002891477,
    -0.024214484,
    -0.019469662,
    -0.038628835,
    -0.05970077,
    0.013868974,
    -0.041725248,
    0.018331435,
    -0.027626732,
    0.047414143,
    0.048645236,
    0.015711607,
    -0.019731916,
    0.02018027,
    0.037067145,
    0.015580378,
    -0.02074777,
    -0.037656497,
    0.0315254,
    0.027829327,
    -0.04953328,
    -0.008974909,
    -0.036621064,
    0.08268924,
    -0.000099023084,
    -0.010808362,
    0.017444545,
    -0.036837447,
    -0.033786334,
    0.024554044,
    0.038338773,
    0.0015833074,
    -0.016416071,
    0.026449595,
    -0.032863718,
    0.020646136,
    -0.005101266,
    0.003269877,
    0.043449853,
    0.026952377,
    0.0030762502,
    -0.043083463,
    0.011685804,
    0.02702897,
    -0.019786451,
    0.09016056,
    -0.019395946,
    0.00547562,
    0.00805198,
    0.027644351,
    0.013978436,
    -0.01701422,
    0.0027872338,
    0.0068438747,
    -0.034153488,
    0.041465588,
    0.0065093203,
    -0.010097714,
    -0.008133783,
    0.033143714,
    -0.017333148,
    0.043355733,
    -0.00871402,
    -0.08563055,
    -0.0106432075,
    0.035572823,
    -0.023785846,
    -0.004012492,
    0.042794853,
    -0.031091385,
    0.0576266,
    0.0070195203,
    0.0765921,
    -0.043606408,
    -0.023182996,
    -0.04500981,
    0.0025196855,
    -0.015288511,
    0.031438597,
    0.051824644,
    -0.042258043,
    0.03338895,
    -0.025437905,
    0.031160489,
    -0.0072392435,
    -0.031922713,
    0.029755816,
    0.012957701,
    -0.10024418,
    0.032848,
    -0.007295161,
    -0.0035617317,
    0.028405763,
    0.061833233,
    -0.04108825,
    -0.020171262,
    0.020549063,
    -0.026362132,
    -0.023915224,
    0.007996089,
    -0.030391349,
    -0.0028575344,
    -0.018893851,
    0.02560692,
    0.008163355,
    0.087368175,
    0.008817629,
    -0.00850316,
    -0.059040304,
    0.041495502,
    -0.040280342,
    0.0068111503,
    0.0021893613,
    0.0022273697,
    0.005985552,
    0.0505196,
    -0.04750377,
    -0.03359202,
    0.020417644,
    0.0044759694,
    -0.0035043096,
    0.048957326,
    -0.011384678,
    0.008479551,
    0.025835814,
    0.0014910818,
    0.039479405,
    0.057871632,
    0.016251555,
    0.06249111,
    -0.019609375,
    -0.04390419,
    0.017925067,
    -0.055957958,
    -0.019112395,
    -0.04754885,
    0.026070505,
    -0.019006608,
    -0.030362992,
    -0.0067993933,
    -0.006561339,
    0.026431026,
    -0.03405452,
    0.057274282,
    -0.026786203,
    0.063958324,
    -0.04453278,
    0.027547128,
    -0.03851104,
    0.01796476,
    0.031259652,
    0.04863174,
    -0.024669012,
    -0.034748323,
    -0.018997308,
    -0.05671984,
    -0.021414421,
    0.020377612,
    -0.030505922,
    -0.0050755935,
    -0.033292443,
    0.002657024,
    -0.038488954,
    0.009190322,
    -0.049295817,
    0.041600667,
    0.0049329526,
    0.0032398892,
    -0.027688216,
    0.060459703,
    0.03917895,
    0.05121542,
    0.011903356,
    0.0094349375,
    -0.01939282,
    -0.040036276,
    0.019289287,
    0.007947662,
    -0.05668005,
    0.01639571,
    0.013513371,
    -0.10730804,
    0.011741851,
    0.052281383,
    0.0060147797,
    -0.0016338177,
    0.016279288,
    0.012750764,
    -0.04507693,
    -0.00013838317,
    -0.020932881,
    0.0028839025,
    -0.0015208381,
    0.0013958535,
    0.023533827,
    -0.010494416,
    -0.061766583,
    -0.02134274,
    -0.022852637,
    0.054971635,
    0.026075963,
    -0.021454506,
    0.02648363,
    -0.030089613,
    0.028793827,
    0.004582815,
    -0.021465372,
    -0.017831849,
    -0.06147862,
    -0.05767438,
    0.09083923,
    -0.05611259,
    0.017855838,
    0.009335081,
    -0.045156814,
    0.06599881,
    0.018773748,
    -0.05827166,
    0.016615061,
    -0.006753534,
    0.01607565,
    0.027570006,
    -0.020239603,
    -0.03056045,
    0.046354145,
    0.03691325,
    0.031975202,
    0.022407934,
    0.025474546,
    -0.045023665,
    -0.040520623,
    0.0005759944,
    -0.03525117,
    -0.009240973,
    -0.011385803,
    0.08493358,
    0.018094597,
    0.035135623,
    0.016993279,
    0.01320788,
    0.07891705,
    -0.020045092,
    -0.033938758,
    -0.0056153582,
    0.03615839,
    -0.031113567,
    0.057805743,
    -0.001218427,
    -0.021837134,
    0.029644802,
    -0.0033356778,
    0.040365815,
    -0.018033424,
    -0.02393337,
    0.05093956,
    0.030515084,
    -0.037502967,
    0.009851229,
    0.0072234045,
    -0.048626166,
    0.037203453,
    0.05917087,
    -0.03617051,
    0.00980295,
    -0.038709767,
    0.02074771,
    0.03775127,
    -0.03192831,
    0.05048824,
    -0.001492862,
    -0.021132791,
    0.08444902,
    -0.03443452,
    -0.040238414,
    0.048974395,
    -0.027845262,
    0.07948588,
    0.0208495,
    0.026636329,
    -0.02487519,
    -0.029094454,
    -0.05993427,
    0.03780091,
    -0.012249043,
    0.0028786385,
    0.043765884,
    -0.028861433,
    0.009502931,
    -0.030093342,
    -0.026304517,
    -0.05845765,
    -0.0392811,
    0.029583348,
    -0.01641044,
    -0.005475869,
    0.04321726,
    -0.06391988,
    -0.025680711,
    -0.0030519557,
    0.030743454,
    -0.0007805563,
    -0.04256318,
    -0.005030573,
    -0.0041211224,
    0.021191673,
    0.04913769,
    0.027602818,
    -0.014373903,
    0.03175012
  ]
},
{
  "_id": "54638",
  "content": "Employee number 54638, name  Jane Johnson, department  Operations, location  Seattle, salary  140000",
  "embedding": [
    0.026050478,
    -0.048555247,
    -0.008828629,
    0.019901045,
    0.034666445,
    0.010111517,
    0.06873206,
    -0.030538522,
    0.009940293,
    0.0339663,
    -0.03364418,
    -0.03269781,
    -0.029471608,
    -0.0027008695,
    -0.02049043,
    -0.0029114042,
    0.010917951,
    -0.028362973,
    -0.0075063263,
    -0.06207452,
    -0.021798516,
    0.0039276425,
    0.00055708154,
    -0.02095584,
    0.01713003,
    -0.01722649,
    0.008000153,
    -0.08242634,
    -0.028559122,
    0.018734612,
    -0.066623494,
    0.034287848,
    -0.03751501,
    0.03244232,
    -0.03420307,
    0.023143344,
    -0.03240853,
    -0.023571912,
    -0.00683183,
    0.03096674,
    -0.014259085,
    -0.011342896,
    -0.026998486,
    0.014940984,
    0.050578054,
    -0.028980328,
    0.023303777,
    -0.004296986,
    0.0293136,
    -0.061096653,
    0.03287692,
    0.015354411,
    0.021088544,
    -0.0015228266,
    0.0029424573,
    -0.082046166,
    0.0040660985,
    0.0074269758,
    -0.010958359,
    -0.035012763,
    -0.018889561,
    -0.0059276978,
    0.002083359,
    0.036374025,
    -0.004819571,
    -0.04908644,
    -0.017455256,
    -0.009606802,
    0.06495625,
    0.0032073925,
    -0.052301414,
    -0.041856,
    0.05783131,
    -0.017050996,
    -0.061963696,
    -0.06982274,
    -0.03540072,
    0.03466075,
    0.055206526,
    0.011832436,
    0.0021956502,
    -0.007684086,
    -0.020636348,
    -0.018782893,
    -0.047240626,
    0.047610387,
    -0.0076809353,
    -0.0100567825,
    -0.025776941,
    0.04181693,
    -0.0046912674,
    -0.012204287,
    0.014001371,
    -0.063043125,
    -0.036930084,
    0.059859123,
    -0.010818763,
    -0.045730297,
    -0.0075380807,
    -0.019068131,
    0.019801125,
    -0.049903613,
    -0.05432268,
    -0.0071445843,
    0.0057908674,
    -0.002666185,
    -0.0075345123,
    -0.019908248,
    -0.00643323,
    -0.00071061886,
    -0.0838654,
    -0.008257412,
    -0.038826358,
    -0.0513455,
    0.072962046,
    -0.020993974,
    -0.049816236,
    0.070966154,
    0.00079034764,
    0.015203719,
    0.05684234,
    0.03761177,
    0.016545977,
    0.020625291,
    0.0210726,
    0.02898525,
    -0.0033419074,
    0.0032714924,
    0.11322761,
    0.054507524,
    -0.04121643,
    -0.039163385,
    0.040570177,
    -0.0072572003,
    0.038773507,
    0.0637184,
    0.069037475,
    -0.028668208,
    0.07188595,
    -0.02913201,
    0.0063090385,
    0.007426714,
    -0.03193378,
    -0.006524865,
    -0.0127412435,
    0.015498198,
    -0.020522788,
    -0.011274028,
    0.028620226,
    0.005763184,
    0.022521585,
    0.0072611654,
    -0.048059847,
    -0.01234606,
    0.04490082,
    0.03871421,
    -0.044477567,
    0.027942147,
    -0.0066905613,
    -0.014890972,
    0.048969653,
    0.019693347,
    -0.021390632,
    0.033437464,
    -0.015723728,
    -0.011988548,
    0.0554776,
    -0.03956305,
    -0.07680316,
    -0.031703744,
    -0.01612956,
    0.04235391,
    -0.070821315,
    0.011926204,
    -0.053500686,
    -0.050267965,
    0.00074491126,
    -0.008569316,
    -0.027631104,
    0.0061910404,
    -0.033930015,
    -0.007472883,
    0.0026893707,
    0.0384716,
    0.018170964,
    0.050014913,
    0.05871329,
    -0.044597052,
    -0.014061176,
    -0.003683495,
    0.025531424,
    0.017034912,
    -0.027470706,
    -0.006491179,
    -0.009489945,
    0.017133964,
    -0.015572295,
    0.0139110815,
    -0.02003761,
    -0.037344135,
    -0.030428719,
    0.06549211,
    0.010248525,
    0.0028726796,
    0.033761874,
    -0.015765324,
    0.046935823,
    -0.04185924,
    -0.006468727,
    0.050379142,
    0.007927611,
    0.03851822,
    0.008543064,
    0.0078075267,
    0.039120134,
    0.03586357,
    0.04638511,
    -0.010768516,
    -0.0067371903,
    -0.025532754,
    -0.0051189596,
    0.02152079,
    -0.04576048,
    0.04305608,
    -0.026554907,
    -0.018880805,
    -0.01480849,
    0.026723232,
    0.001033573,
    -0.037126005,
    -0.0020229125,
    0.040790092,
    0.0018653974,
    -0.049913183,
    0.04781728,
    0.027713154,
    0.010776397,
    0.01318502,
    -0.026499385,
    -0.009594083,
    -0.03862901,
    0.016218811,
    0.06014656,
    0.025734013,
    -0.0347474,
    0.0116609605,
    -0.038452834,
    0.0052867737,
    0.010370307,
    0.027156852,
    -0.0015308461,
    -0.06445644,
    0.050907042,
    0.022530565,
    -0.048766818,
    0.05991084,
    -0.09708642,
    -0.009711094,
    0.015859898,
    -0.029318294,
    0.050506905,
    0.00735409,
    -0.0138486065,
    0.024359517,
    -0.034031246,
    0.024766166,
    -0.01821558,
    -0.06686822,
    -0.0487108,
    0.052032128,
    0.016451957,
    -0.03551824,
    0.08024126,
    0.051618367,
    0.013979535,
    0.010763741,
    -0.023814084,
    0.04610131,
    0.008492314,
    -0.06802373,
    -0.015461633,
    0.055361446,
    0.0074120234,
    -0.04642445,
    -0.028934892,
    0.01169551,
    -0.030712495,
    0.0039937347,
    0.008349997,
    -0.06625405,
    -0.08341982,
    0.013278654,
    -0.026365193,
    -0.039572082,
    -0.064723566,
    0.0052512945,
    -0.02465726,
    0.025906282,
    0.005685407,
    0.006141651,
    0.0044314065,
    -0.039461534,
    -0.032410044,
    -0.074979536,
    0.05221336,
    -0.010037091,
    -0.00015792609,
    -0.070082806,
    -0.0063522724,
    -0.036909256,
    0.024703242,
    -0.025880957,
    -0.03181115,
    0.025542444,
    0.020821305,
    0.05083042,
    0.00440165,
    -0.017547268,
    -0.038332768,
    0.06295742,
    -0.003380332,
    0.0017819487,
    0.031406853,
    -0.03936085,
    -0.014774891,
    0.05555366,
    0.0013044683,
    0.071219094,
    0.0027098448,
    0.0090771,
    0.004294718,
    0.04097738,
    0.0038274624,
    0.09351304,
    0.01993581,
    0.03123765,
    -0.062362995,
    0.017761108,
    -0.06349266,
    -0.023149393,
    0.041712813,
    0.023032729,
    -0.046279017,
    -0.059766676,
    0.013576986,
    -0.035526287,
    0.0009959425,
    0.042815145,
    0.052038774,
    0.047260206,
    -0.015755137,
    0.011777429,
    -0.013718928,
    -0.018773504,
    -0.041940242,
    -0.016540648,
    0.056323376,
    -0.009581614,
    0.012827593,
    0.002802622,
    -0.047416028,
    0.06029572,
    -0.026624044,
    -0.0059878556,
    -0.01112658,
    0.0064357584,
    0.015744798,
    0.0027346082,
    -0.013077302,
    0.027371943,
    0.028480768,
    -0.029083466,
    -0.016170066,
    0.018732633,
    -0.02920547,
    -0.049596816,
    -0.050539367,
    -0.023739604,
    -0.016439682,
    0.023610277,
    0.03793149,
    -0.01936672,
    0.00054942124,
    0.03477947,
    0.022074739,
    -0.008824361,
    -0.016267285,
    0.032433596,
    -0.026371641,
    0.06440936,
    0.016472073,
    -0.012704358,
    0.12420736,
    -0.0101508675,
    0.023653913,
    -0.036456037,
    -0.009319963,
    -0.02745349,
    0.011565427,
    -0.016726809,
    -0.00910894,
    -0.027309556,
    -0.020953115,
    -0.0004489086,
    -0.017622823,
    -0.026881404,
    -0.016441276,
    -0.028333068,
    0.051373687,
    -0.06849969,
    0.048643496,
    0.06129681,
    0.043112013,
    -0.10503493,
    -0.032169662,
    -0.016303875,
    0.038900618,
    0.017744469,
    -0.0078635495,
    0.011909131,
    -0.02633716,
    0.012839195,
    -0.034210175,
    -0.049735658,
    -0.012025739,
    0.024162453,
    -0.017275587,
    -0.027206033,
    -0.01216894,
    0.056151856,
    -0.002266644,
    -0.017719636,
    -0.024023125,
    0.012112513,
    0.035618242,
    -0.066973604,
    -0.038296815,
    0.046060294,
    -0.03857978,
    0.04528379,
    0.043774627,
    -0.025222767,
    0.08498407,
    0.0023034313,
    -0.007176461,
    -0.018639492,
    0.023903845,
    -0.040975634,
    0.014675711,
    -0.08696412,
    -0.0012359321,
    -0.060993966,
    -0.024204629,
    -0.045965314,
    -0.05470206,
    -0.0545379,
    0.031496808,
    0.059156094,
    0.009160291,
    0.0072219935,
    -0.006173258,
    -0.03295994,
    -0.01801206,
    -0.057803974,
    0.041660473,
    -0.039624795,
    0.028510751,
    -0.030607404,
    0.06521628,
    0.06455515,
    0.013236343,
    -0.013641919,
    0.03251663,
    0.019429607,
    0.020611761,
    -0.047394972,
    -0.033903588,
    0.030801337,
    0.03389709,
    -0.033200398,
    -0.00968848,
    -0.042483523,
    0.062307518,
    -0.024104252,
    -0.038019832,
    0.037520684,
    -0.02434741,
    -0.015609218,
    0.0065647936,
    0.043396086,
    0.014070153,
    0.0043344726,
    0.024882345,
    -0.022135641,
    -0.01799605,
    0.00038329684,
    -0.01741619,
    0.044463806,
    0.031136844,
    0.032308206,
    0.0007304428,
    0.035526954,
    0.028219154,
    -0.021445524,
    0.059003815,
    -0.0415958,
    -0.0043805623,
    -0.0006041634,
    0.028271,
    0.038045794,
    -0.007856862,
    -0.0030909725,
    -0.032664094,
    -0.017286232,
    0.024400914,
    -0.013834741,
    -0.02652701,
    0.029634649,
    0.01700266,
    -0.03734089,
    0.038840823,
    -0.016492758,
    -0.093306154,
    -0.0044792993,
    0.04936495,
    -0.016763058,
    0.024115685,
    0.05415202,
    -0.04315434,
    0.015969714,
    -0.021037051,
    0.05564539,
    -0.055493116,
    -0.02337645,
    -0.025281547,
    -0.0065010595,
    0.008250707,
    0.055807795,
    0.02414763,
    -0.022564175,
    0.02834781,
    -0.040628407,
    0.026874747,
    -0.01892121,
    -0.035771616,
    0.0018971186,
    0.030991131,
    -0.058407318,
    0.022544177,
    -0.0099294195,
    -0.003015739,
    0.009533158,
    0.06220805,
    -0.0018919198,
    -0.015758067,
    0.0045811604,
    -0.022871247,
    -0.026954936,
    -0.007922866,
    -0.015751228,
    -0.024009718,
    -0.019846817,
    0.00086579495,
    0.007355841,
    0.07742838,
    0.039498612,
    0.0018927547,
    -0.049126364,
    0.035092838,
    -0.038997784,
    0.014563989,
    0.0014772905,
    -0.0058927247,
    0.0004669789,
    0.054502,
    -0.0101958765,
    -0.06462403,
    0.030232383,
    0.015468133,
    0.008747526,
    0.058180943,
    0.009895715,
    -0.023807824,
    0.016150286,
    -0.033176575,
    0.03896909,
    0.07366637,
    -0.02589846,
    0.081121355,
    0.012857258,
    -0.0320698,
    0.026557323,
    -0.05475815,
    -0.045223676,
    -0.006914698,
    0.059804097,
    -0.030856598,
    -0.02270003,
    -0.006721817,
    0.007834129,
    0.014207969,
    -0.04459566,
    0.060735647,
    -0.004343931,
    0.06916978,
    -0.025479676,
    0.001981753,
    -0.049986716,
    0.019160662,
    0.019025618,
    0.03463173,
    -0.0051061907,
    -0.039971903,
    -0.009554134,
    -0.051973134,
    -0.02322696,
    0.000666707,
    0.0076026963,
    0.0070748134,
    -0.027127214,
    0.0021439027,
    -0.0380424,
    0.019361308,
    -0.021825459,
    0.024104213,
    -0.001751936,
    0.020918885,
    -0.0037656801,
    0.053490546,
    0.072466716,
    0.06307481,
    0.025220444,
    0.030272197,
    -0.011731547,
    -0.056140322,
    0.048621643,
    -0.022496052,
    -0.06380631,
    0.038866386,
    -0.010590717,
    -0.108824804,
    0.006726385,
    0.050433647,
    0.0057739234,
    -0.01420325,
    0.023737092,
    0.042579204,
    -0.053798538,
    -0.018823216,
    -0.03827396,
    -0.015979653,
    -0.0069397204,
    0.02125115,
    0.0033473414,
    -0.013927182,
    -0.05654852,
    -0.019954465,
    -0.035381112,
    0.056015182,
    0.0065556956,
    -0.023087364,
    0.043700524,
    0.0027661035,
    0.025146691,
    0.012687644,
    -0.027370248,
    -0.0010651633,
    -0.06115836,
    -0.029139342,
    0.08521571,
    -0.042291123,
    0.030378494,
    0.014432462,
    -0.046826813,
    0.050151218,
    0.045153998,
    -0.04796362,
    0.022682115,
    0.011545504,
    0.020556057,
    -0.0042003356,
    -0.021686615,
    -0.0012217021,
    0.019806935,
    0.047820672,
    0.05978573,
    0.01758219,
    0.02204856,
    -0.039568268,
    -0.021229789,
    0.014548975,
    -0.034249913,
    -0.023128428,
    -0.011201689,
    0.07169892,
    0.026005901,
    0.028781159,
    0.0130280135,
    0.03135926,
    0.0843804,
    -0.0062047434,
    -0.061045166,
    0.010564766,
    0.018106297,
    -0.043597803,
    0.02699747,
    -0.0120446915,
    -0.024967365,
    0.034383804,
    0.0030797508,
    0.03732648,
    -0.06742948,
    -0.041975114,
    0.036880396,
    0.025295557,
    0.016465476,
    0.028427439,
    -0.004287951,
    -0.008966516,
    0.03698348,
    0.05051995,
    -0.024159456,
    0.0052558105,
    -0.04339689,
    0.009192153,
    0.045353524,
    -0.049608096,
    0.06556983,
    0.0049719103,
    0.000014385518,
    0.06860882,
    -0.03174607,
    -0.0073257447,
    0.0044105197,
    -0.03662372,
    0.09250486,
    0.04256795,
    0.031364053,
    0.0016935823,
    -0.0680488,
    -0.07860276,
    0.024094777,
    0.016591892,
    0.022528214,
    0.029432567,
    -0.02577635,
    0.013138877,
    -0.021803275,
    -0.030736644,
    -0.064376354,
    -0.0935471,
    0.018339334,
    -0.040749416,
    0.036033116,
    0.040105127,
    -0.059686333,
    -0.028928738,
    -0.0044766283,
    0.030114502,
    0.018945087,
    -0.029741868,
    0.0052247434,
    -0.013826671,
    0.06707814,
    0.0406519,
    0.03318739,
    0.010909002,
    0.029758368
  ]
}])




InsertManyResult(['54634', '54633', '54636', '54635', '54637', '54638'], acknowledged=True)

### MongoDB Atlas Vector Search with Gemini 2.0

A vector similarity search implementation that leverages MongoDB Atlas Vector Search and Google's Gemini 2.0 embeddings to perform semantic document searches, returning the k-most similar documents based on query embedding comparison.

In [29]:


from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import MongoDBAtlasVectorSearch
import os

# Assuming you have set your MongoDB connection string as an environment variable
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector_store = MongoDBAtlasVectorSearch.from_connection_string(
      connection_string=MONGO_URI,
      namespace="google-ai.embedded_docs",
      embedding_key="embedding",
      text_key="content",
      index_name="vector_index",
      embedding=embeddings
  )
def atlas_search(query: str, k: int = 5):
    """
    Perform a vector similarity search using MongoDB Atlas Vector Search.
    """
    try:

        vector_search_results = vector_store.similarity_search_with_score(query=query, k=k)
        ## Remove "embedding" key
        modified_results = []
        for doc, score in vector_search_results:
            if "embedding" in doc.metadata:
                del doc.metadata["embedding"]
            modified_results.append((doc, score))
        return modified_results

    except Exception as e:
        print(f"An error occurred: {e}")
        return []

Additionally, including a function to create new teams with specified members as a document inside the Atlas database.

In [47]:
# prompt: I need 2 tools one that will use MongoDB pipeline input and query the "ai_shop" db and "products" collection and the the second will create orders in the "orders" collection


teams_collection = db["team"]



@retry.Retry()
def create_team(name, people):
  """
  Creates a new team in the teams collection.

  Args:
    name : Name of the team
    people : A list of people in the team.

  Returns:
    A message indicating whether the order was successfully created or an error message.
  """
  try:
      result = teams_collection.insert_one({'name': name,
                                            'people' : people

      })
      return f"Team created successfully with ID: {result.inserted_id}"
  except Exception as e:
      return f"Error creating order: {e}"

tool_calls = {
    'atlas_search_tool': atlas_search,
    'create_order': create_team
}



Lets create the tool defenitions

In [61]:
team_tool = {
    "name": "create_team",
    "description": "Creates a new team in the teams collection.",
    "parameters": {
        "type": "object",
        "properties": {
            "team_data": {
                "type": "object",
                "description": "A dictionary containing the team details.",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "team name"
                    },
                    "people": {
                        "type": "array",
                        "description": "A list of people in the team.",
                        "items": {
                            "type": "string",
                            "description": "A person in the team."
                        }
                    }
                },
                "required": ["name","people"]
            }
        },
        "required": ["team_data"]
    }
}

atlas_search_tool = {
    "name": "atlas_search_tool",
    "description": " Perform a vector similarity search for employees using MongoDB Atlas Vector",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "The search query."
            },
            "k": {
                "type": "integer",
                "description": "The number of results to return."
            }
        },
        "required": ["query"]
    }
}


tools = [
    {'function_declarations': [team_tool, atlas_search_tool]}
]

tool_calls = {
    'atlas_search_tool': atlas_search,
    'create_team': create_team
}

We will first search for "females" similarity search in our Employee database using the "AUDIO" modality response to recieve a voice based response.

In [64]:
prompt = """ Search for 'Human Resources' employees only.
"""



await run(prompt, tools=tools, modality = "AUDIO")

before client invoke [{'function_declarations': [{'name': 'create_team', 'description': 'Creates a new team in the teams collection.', 'parameters': {'type': 'object', 'properties': {'team_data': {'type': 'object', 'description': 'A dictionary containing the team details.', 'properties': {'name': {'type': 'string', 'description': 'team name'}, 'people': {'type': 'array', 'description': 'A list of people in the team.', 'items': {'type': 'string', 'description': 'A person in the team.'}}}, 'required': ['name', 'people']}}, 'required': ['team_data']}}, {'name': 'atlas_search_tool', 'description': ' Perform a vector similarity search for employees using MongoDB Atlas Vector', 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'The search query.'}, 'k': {'type': 'integer', 'description': 'The number of results to return.'}}, 'required': ['query']}}]}]


 Search for 'Human Resources' employees only.


-------------------------------

Before tool call function_calls=[FunctionCall(id='function-call-7239458625166350317', args={'query': 'Human Resources'}, name='atlas_search_tool')]

>>>  function_responses=[FunctionResponse(id='function-call-7239458625166350317', name='atlas_search_tool', response={'result': [(Document(metadata={'_id': '54637'}, page_content='Employee number 54637, name  John Johnson, department  HR, location  Miami, salary  110000'), 0.841124415397644), (Document(metadata={'_id': '54634'}, page_content='Employee number 54634, name  Jane Doe, department  Marketing, location  Los Angeles, salary  120000'), 0.8330270051956177), (Document(metadata={'_id': '54636'}, page_content='Employee number 54636, name  Jane Smith, department  Finance, location  Chicago, salary  130000'), 0.8256025910377502), (Document(metadata={'_id': '54638'}, page_content='Employee number 54638, name  Jane Johnson, department  Operations, location  Seattle, salary  140000'), 0.8211219310760498), (Document(metadata={'_id': '54633'}

Now, lets use the TEXT modality to perform a complex task for finding and creating a team from only the marketing employees.

In [65]:
tools = [ {'code_execution': {}},
         {'function_declarations': [team_tool, atlas_search_tool]}
          ]

prompt = """Search for "marketing" in the database and use thier names to create a team :
1. Search for "marketing"
2. Take the located marketing employees to a team called "Marketing Working group".

"""



await run(prompt, tools=tools, modality = "TEXT")

before client invoke [{'code_execution': {}}, {'function_declarations': [{'name': 'create_team', 'description': 'Creates a new team in the teams collection.', 'parameters': {'type': 'object', 'properties': {'team_data': {'type': 'object', 'description': 'A dictionary containing the team details.', 'properties': {'name': {'type': 'string', 'description': 'team name'}, 'people': {'type': 'array', 'description': 'A list of people in the team.', 'items': {'type': 'string', 'description': 'A person in the team.'}}}, 'required': ['name', 'people']}}, 'required': ['team_data']}}, {'name': 'atlas_search_tool', 'description': ' Perform a vector similarity search for employees using MongoDB Atlas Vector', 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'The search query.'}, 'k': {'type': 'integer', 'description': 'The number of results to return.'}}, 'required': ['query']}}]}]


Search for "marketing" in the database and use thier names to create a team :
1. Search for "marketing"
2. Take the located marketing employees to a team called "Marketing Working group".



-------------------------------

-------------------------------

``` python
marketing_employees = default_api.atlas_search_tool(query="marketing")
print(marketing_employees)

```

...

```
{'result': 'Team created successfully with ID: 676acb7c759477c2fbaf03f5'}

```

-------------------------------

OK

. I have created the "Marketing Working group" team with Jane Doe as a

 member.


The function calling feature of the API Can handle a wide variety of functions. Support in the SDK is still under construction. So keep this simple just send a minimal function definition: Just the function's name.

Note that in the live API function calls are independent of the chat turns. The conversation can continue while a function call is being processed.